# Import

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.4f}'.format
import pandas_ta as ta
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import datetime

%matplotlib inline

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# test a a aa

### tset bbb

# Declare global variable:
    

In [ ]:
log_return_masking = ['LogReturn_1bar','LogReturn_3bar','LogReturn_10bar','LogReturn_20bar']
pip_return_masking = ['PipReturn_1bar','PipReturn_3bar','PipReturn_5bar','PipReturn_10bar',
                      'PipReturn_15bar','PipReturn_20bar']

# Functions

## Generate points Functions

### Generate pivotID Function

In [ ]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

def RSIpivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.RSI[l]>df1.RSI[i]):
            pividlow=0
        if(df1.RSI[l]<df1.RSI[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0 
    
def MACDpivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.MACD[l]>df1.MACD[i]):
            pividlow=0
        if(df1.MACD[l]<df1.MACD[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0 

### Generate pointPosition to graph Function

In [ ]:
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-4
    elif x['pivot']==2:
        return x['high']+1e-4
    else:
        return np.nan

def RSIpointpos(x):
    if x['RSIpivot']==1:
        return x['RSI']-1e-4
    elif x['RSIpivot']==2:
        return x['RSI']+1e-4
    else:
        return np.nan
    
def MACDpointpos(x):
    if x['MACDpivot']==1:
        return x['MACD']-1e-4
    elif x['MACDpivot']==2:
        return x['MACD']+1e-4
    else:
        return np.nan

### Generate price slope for each bar

In [ ]:
def slopeGenerate(df, x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    closp = np.array([])
    xxclos = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        closp = np.append(closp, df.iloc[i].close)
        xxclos = np.append(xxclos, i)

    slclos, interclos = np.polyfit(xxclos, closp,1)
    
    return slclos

### Generate divergence signal Function

In [ ]:
# RSi ------------------
def divsignalRSI(df, x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].RSIpivot == 1:
            minimRSI = np.append(minimRSI, df.iloc[i].RSI)
            xxminRSI = np.append(xxminRSI, df.iloc[i].name)
        if df.iloc[i].RSIpivot == 2:
            maximRSI = np.append(maximRSI, df.iloc[i].RSI)
            xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    if maxim.size<2 or minim.size<2 or maximRSI.size<2 or minimRSI.size<2:
        return 0
    
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI,1)
    slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI,1)
    
    
    if slmin > 1e-4 and slmax > 1e-4 and slmaxRSI <-0.1:
        return 1
    elif slmin < -1e-4 and slmax < -1e-4 and slminRSI > 0.1:
        return 2
    else:
        return 0
# MACD---------------------------------
    
def divsignalMACD(df, x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximMACD = np.array([])
    minimMACD = np.array([])
    xxminMACD = np.array([])
    xxmaxMACD = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].MACDpivot == 1:
            minimMACD = np.append(minimMACD, df.iloc[i].MACD)
            xxminMACD = np.append(xxminMACD, df.iloc[i].name)
        if df.iloc[i].MACDpivot == 2:
            maximMACD = np.append(maximMACD, df.iloc[i].MACD)
            xxmaxMACD = np.append(xxmaxMACD, df.iloc[i].name)

    if maxim.size<2 or minim.size<2 or maximMACD.size<2 or minimMACD.size<2:
        return 0
    
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    slminMACD, intercminMACD = np.polyfit(xxminMACD, minimMACD,1)
    slmaxMACD, intercmaxMACD = np.polyfit(xxmaxMACD, maximMACD,1)
    
    
    if slmin > 1e-4 and slmax > 1e-4 and slmaxMACD <-0.1:
        return 1
    elif slmin < -1e-4 and slmax < -1e-4 and slminMACD > 0.1:
        return 2
    else:
        return 0

### Generate div signal 2 (without slopes of indicator piv point) Function

##### MACD

In [ ]:
def divsignal2MACD(df, x, nbackcandles, startRange):
    backcandles=nbackcandles 
    candleid = int(x.name)

    closp = np.array([])
    xxclos = np.array([])
    
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximMACD = np.array([])
    minimMACD = np.array([])
    xxminMACD = np.array([])
    xxmaxMACD = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        closp = np.append(closp, df.iloc[i].close)
        xxclos = np.append(xxclos, i)
        
    for i in range(candleid-backcandles-startRange, candleid+1 - startRange):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].MACDpivot == 1:
            minimMACD = np.append(minimMACD, df.iloc[i].MACD)
            xxminMACD = np.append(xxminMACD, df.iloc[i].name)
        if df.iloc[i].MACDpivot == 2:
            maximMACD = np.append(maximMACD, df.iloc[i].MACD)
            xxmaxMACD = np.append(xxmaxMACD, df.iloc[i].name)

    slclos, interclos = np.polyfit(xxclos, closp,1)
    
    if slclos > 1e-4 and (maximMACD.size<2 or maxim.size<2):
        return 0
    if slclos < -1e-4 and (minimMACD.size<2 or minim.size<2):
        return 0
# signal decisions here !!!
    if (slclos > 1e-4):
        if maximMACD[-1]<maximMACD[-2] and maxim[-1]>maxim[-2]:
            return 1
    elif (slclos < -1e-4):
        if minimMACD[-1]>minimMACD[-2] and minim[-1]<minim[-2]:
            return 2
    else:
        return 0

##### RSI

In [ ]:
def divsignal2RSI(df, x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    closp = np.array([])
    xxclos = np.array([])
    
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        closp = np.append(closp, df.iloc[i].close)
        xxclos = np.append(xxclos, i)
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].RSIpivot == 1:
            minimRSI = np.append(minimRSI, df.iloc[i].RSI)
            xxminRSI = np.append(xxminRSI, df.iloc[i].name)
        if df.iloc[i].RSIpivot == 2:
            maximRSI = np.append(maximRSI, df.iloc[i].RSI)
            xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    slclos, interclos = np.polyfit(xxclos, closp,1)
    
    if slclos > 1e-4 and (maximRSI.size<2 or maxim.size<2):
        return 0
    if slclos < -1e-4 and (minimRSI.size<2 or minim.size<2):
        return 0
# signal decisions here !!!
    if slclos > 1e-4:
        if maximRSI[-1]<maximRSI[-2] and maxim[-1]>maxim[-2]:
            return 1
    elif slclos < -1e-4:
        if minimRSI[-1]>minimRSI[-2] and minim[-1]<minim[-2]:
            return 2
    else:
        return 0

### Generate div signal 3 (without slopes of anything) Function

##### MACD

In [ ]:
def divsignal3MACD(df, x, nbackcandles, startRange, thresholdRSI_up, thresholdRSI_down):
    '''
    startRange: number of bar in one side of the compared range of the pivor, ig. pivot (10,10)=> startRange = 10
    '''
    backcandles=nbackcandles 
    candleid = int(x.name)
 
    maxim = np.array([])
    minim = np.array([])
    # xxmin = np.array([])
    # xxmax = np.array([])

    maximMACD = np.array([])
    minimMACD = np.array([])
    # xxminMACD = np.array([])
    # xxmaxMACD = np.array([])

    for i in range(candleid-backcandles - startRange, candleid+1 - startRange):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            # xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            # xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].MACDpivot == 1:
            minimMACD = np.append(minimMACD, df.iloc[i].MACD)
            # xxminMACD = np.append(xxminMACD, df.iloc[i].name)
        if df.iloc[i].MACDpivot == 2:
            maximMACD = np.append(maximMACD, df.iloc[i].MACD)
            # xxmaxMACD = np.append(xxmaxMACD, df.iloc[i].name)
   
    if minimMACD.size< 2 or minim.size<2:
        return 0
    if maximMACD.size< 2 or maxim.size<2:
        return 0
    
# signal decisions here !!!

    if x.RSI > thresholdRSI_up and maximMACD[-1] < maximMACD[-2] and maxim[-1] > maxim[-2]:
        return 1
    elif x.RSI < thresholdRSI_down and minimMACD[-1] > minimMACD[-2] and minim[-1] < minim[-2]:
        return 2
    else:
        return 0

## Trade adjustment functions

#### Eliminate consecutive trades function

In [ ]:
def elimConsTrades(df, elim_return_type = pip_return_masking):
    '''
    return_type = 'log' or 'pip'
    elim_return_type = pip_return_masking or log_return_masking
    '''
    
    df =  df.reset_index()
    
    for i in df['index'][:-3].index:
        if df['index'][i+1] == (df['index'][i] +1):
            df['index'][i+1] = 0
        if df['index'][i+2] == (df['index'][i] +2):
            df['index'][i+2] = 0
        if df['index'][i+3] == (df['index'][i] +3):
            df['index'][i+3] = 0
                      
    elim_stat_df = df[df['index']!= 0][elim_return_type].describe().T
    elim_stat_df['Sharpe ratio'] = elim_stat_df['mean']/elim_stat_df['std']
    elim_stat_df['Count x Sharpe'] = elim_stat_df['count']*elim_stat_df['Sharpe ratio']        
      
    elim_result_df = df[df['index']!= 0]

    return elim_result_df, elim_stat_df

## Test Function

#### falsebreak_wDiv_test(): ===> False_break_test Functions

In [ ]:
# ver 2
def falsebreak_wDiv_test(dfpl,side, fwdCandle, div_indi = "MACD", falsebreak_bar = 0, 
                         ATR_surpass = 0, ATR_commpare_range = 0, div_signal = 'divSignal2MACD',
                         elim_cons_trade = True, return_type = pip_return_masking):
    '''
    side: 'buy' or 'sell'
    Test false break signal on divergence
    
    div_indi : indicator used for divergence signal.
    
    ATR_surpass: distance between high of break bar and currHigh as multipler of ATR
    
    falsebreak_bar: number of bars after false break considered to be signal
    
    fwdCandle: num of candles to add to Pivot before considered currHigh/Low
    
    div_signal: type of divergence signal: divSignalMACD, divSignal2MACD, divSignal3MACD, or 'No' to not use divergence vv..
    
    elim_cons_trade: eliminate consecutive trades, one trade per time
    
    return_type = pip_return_masking or log_return_masking
    '''
    dfpl['CurrLow'] = 0
    dfpl['CurrHigh'] = 0
    fwdCandle = fwdCandle
    for i in dfpl['pivot'].index:
        if dfpl['pivot'][i] == 1:
            dfpl['CurrLow'][i+fwdCandle:] = dfpl.low[i]
        elif dfpl['pivot'][i] == 2:
            dfpl['CurrHigh'][i+fwdCandle:] = dfpl.high[i]
    dfpl[f'maxHi{fwdCandle}bar'] = dfpl.high.rolling(fwdCandle).max()
    dfpl[f'maxLo{fwdCandle}bar'] = dfpl.low.rolling(fwdCandle).min()
 
           
    # ----Sell side------------------
        
    if side.lower() == "sell":    
        
        false_break_masking_0bar =  (dfpl.high > dfpl.CurrHigh) & (
                                    dfpl.close < dfpl.CurrHigh) & (
                                    dfpl.high == dfpl[f'maxHi{fwdCandle}bar']) & (
                                    dfpl.UpperWick > ATR_commpare_range * dfpl['ATR']) & (
                                    # dfpl.UpperWick >dfpl.OpenCloseRange) & ( # adding this condition refine the result but too few trades
                                    # abs(dfpl.high - dfpl.low) >= ATR_commpare_range*dfpl['ATR']) & (
                                    # dfpl.close > dfpl.EMA20) & (
                                    # dfpl.open > dfpl.EMA20) & (
                                    # dfpl.close < dfpl.open) & (
                                    # abs(dfpl.high - dfpl.close) > abs(dfpl.close - dfpl.open)) & (

                                    
                                    dfpl.high.shift(periods = 1) < dfpl.CurrHigh) & (
                                    dfpl.HiLowRange.shift(periods = 1) < dfpl.HiLowRange) & (
                                    # dfpl.close.shift(periods = 1) < dfpl.CurrHigh) & (
                                    # dfpl.open.shift(periods = 1) < dfpl.CurrHigh) & (
            
                                    dfpl.high.shift(periods = 2) < dfpl.CurrHigh) & (
                                    dfpl.HiLowRange.shift(periods = 2) < dfpl.HiLowRange) & (
                                    # dfpl.close.shift(periods = 2) < dfpl.CurrHigh) & (
                                    # dfpl.open.shift(periods = 2) < dfpl.CurrHigh) & (
                                    # dfpl.close.shift(periods = 3) < dfpl.CurrHigh) & (
                                    # dfpl.close < dfpl.open.shift(periods = 1)) & (
            
                                    abs(dfpl.high-dfpl.CurrHigh) >= ATR_surpass * dfpl['ATR'])
                                    # abs(dfpl.high - dfpl.close) > abs(dfpl.open - dfpl.close))  & (
        
        false_break_masking_1bar =  (dfpl.open > dfpl.CurrHigh) & (
                                    dfpl.close < dfpl.CurrHigh) & (                                    
                                    dfpl.close < (dfpl.high.shift(periods = 1) - dfpl.low.shift(periods = 1))/2) & (
            
                                    dfpl.open.shift(periods = 1) <dfpl.CurrHigh) & (
                                    dfpl.close.shift(periods = 1) >dfpl.CurrHigh) & (
                                    dfpl.high.shift(periods = 1) == dfpl[f'maxHi{fwdCandle}bar']) & (
                                    
                                    dfpl.high.shift(periods = 2) < dfpl.CurrHigh) & (
                                    # dfpl.close.shift(periods = 2) < dfpl.CurrHigh) & (
                                    # dfpl.open.shift(periods = 2) < dfpl.CurrHigh) & (
            
                                    dfpl.high.shift(periods = 3) < dfpl.CurrHigh) & ( 
                                    # dfpl.close.shift(periods = 3) < dfpl.CurrHigh) & (
                                    # dfpl.open.shift(periods = 3) < dfpl.CurrHigh) 
                                    # dfpl.close.shift(periods = 4) < dfpl.CurrHigh)
                                    abs(dfpl.high.shift(periods = 1)-dfpl.CurrHigh) >= ATR_surpass * dfpl['ATR']) 

        
        if falsebreak_bar == 0:
            false_break_masking = false_break_masking_0bar
        elif falsebreak_bar == 1:
            false_break_masking = false_break_masking_1bar 
        elif falsebreak_bar == 100:
            false_break_masking = false_break_masking_0bar | false_break_masking_1bar
        else :
            warnings.warn("Warning : maximum falsebreak_bar is currrently <= 2 ")
            
        if div_signal.lower() == 'no':
            masking =  false_break_masking
        else:
            divSignal_masking  = dfpl[div_signal] == 1
            masking = divSignal_masking & false_break_masking        
               
        
        #  -------------reuslt----------------------
        
        result_dfpl = dfpl[masking]        
        stat_df = result_dfpl[return_type].describe().T
        stat_df['Sharpe ratio'] = stat_df['mean']/stat_df['std']
        stat_df['Count x Sharpe'] = stat_df['count']*stat_df['Sharpe ratio']
        
# ----Buy side-------------------------------------------------------

    elif side.lower() == 'buy':
           
        false_break_masking_0bar =  (dfpl.low < dfpl.CurrLow) & (
                                    dfpl.close > dfpl.CurrLow) & (
                                    dfpl.low == dfpl[f'maxLo{fwdCandle}bar']) & (
                                    # dfpl.LowerWick >dfpl.OpenCloseRange) & ( # adding this condition refine the result but too few trades
                                    dfpl.LowerWick > ATR_commpare_range * dfpl['ATR']) & (
                                    # abs(dfpl.high - dfpl.low) >= ATR_commpare_range*dfpl['ATR']) & (
            
                                    # dfpl.close < dfpl.EMA20) & ( 
                                    # dfpl.open < dfpl.EMA20) & ( 
                                    # dfpl.close > dfpl.open) & (                                    
                                    # abs(dfpl.low - dfpl.close) > abs(dfpl.close - dfpl.open)) & (
                                    # dfpl.close > dfpl.open) & (
                                    
                                    dfpl.low.shift(periods = 1) > dfpl.CurrLow) & (
                                    dfpl.HiLowRange.shift(periods = 1) < dfpl.HiLowRange) & (
                                    # dfpl.close.shift(periods = 1) > dfpl.CurrLow) & (
                                    # dfpl.open.shift(periods = 1) > dfpl.CurrLow) & (
            
                                    dfpl.low.shift(periods = 2) > dfpl.CurrLow) & (
                                    dfpl.HiLowRange.shift(periods = 2) < dfpl.HiLowRange) & (
                                    # dfpl.close.shift(periods = 2) > dfpl.CurrLow) & (
                                    # dfpl.open.shift(periods = 2) > dfpl.CurrLow) & (
                                    # dfpl.close.shift(periods = 3) > dfpl.CurrLow) & (
                                    # dfpl.close > dfpl.open.shift(periods = 1)) & (
            
                                    abs(dfpl.low - dfpl.CurrLow) >= ATR_surpass * dfpl['ATR']) 
                                    # abs(dfpl.low - dfpl.close) > abs(dfpl.open - dfpl.close))& (
                                    
                    
        false_break_masking_1bar = (dfpl.open < dfpl.CurrLow) & (
                                    dfpl.close > dfpl.CurrLow) & (
                                    dfpl.close > (dfpl.high.shift(periods = 1) - dfpl.low.shift(periods = 1))/2) & (
                                    # dfpl.close > dfpl.open.shift(periods = 1)) & (                      


                                    dfpl.open.shift(periods = 1) > dfpl.CurrLow) & (
                                    dfpl.close.shift(periods = 1) < dfpl.CurrLow) & (
                                    dfpl.low.shift(periods = 1) == dfpl[f'maxLo{fwdCandle}bar']) & (
            
                                    dfpl.low.shift(periods = 2) > dfpl.CurrLow) & (
                                    # dfpl.close.shift(periods = 2) > dfpl.CurrLow) & (
                                    # dfpl.open.shift(periods = 2) > dfpl.CurrLow) & (
                            
                                    dfpl.low.shift(periods = 3) > dfpl.CurrLow) & (
                                    # dfpl.close.shift(periods = 3) > dfpl.CurrLow) & (
                                    # dfpl.open.shift(periods = 3) > dfpl.CurrLow) & (
            
                                    # dfpl.close.shift(periods = 4) > dfpl.CurrLow) & (
                                    abs(dfpl.low.shift(periods = 1) - dfpl.CurrLow) >= ATR_surpass * dfpl['ATR'])
        
        if falsebreak_bar == 0:
            false_break_masking = false_break_masking_0bar
        elif falsebreak_bar == 1:
            false_break_masking = false_break_masking_1bar 
        elif falsebreak_bar == 100:
            false_break_masking = false_break_masking_0bar | false_break_masking_1bar
        else :
            warnings.warn("Warning : maximum falsebreak_bar is currrently <= 2 ")  
            
        if div_signal.lower() == "no":
            masking =  false_break_masking
        else:
            divSignal_masking  = dfpl[div_signal] == 2
            masking = divSignal_masking & false_break_masking   
        
        #-------------reuslt----------------------

        result_dfpl = dfpl[masking] # result_df of the test         
        stat_df = result_dfpl[return_type].describe().T
        stat_df['Sharpe ratio'] = stat_df['mean']/stat_df['std']
        stat_df['Count x Sharpe'] = stat_df['count']*stat_df['Sharpe ratio'] # stat df of the test
        
#         -------eliminate consecutive trades------------
        
    print ( f'result of {side} falsebreak with falsebreak bar = {falsebreak_bar},\n, ATR_Surpass = {ATR_surpass}, div_signal = {div_signal}, elim_cons_trade = {elim_cons_trade},return_type = {return_type} ')
    
    if elim_cons_trade:        
        elim_result_df, elim_stat_df = elimConsTrades(result_dfpl, elim_return_type = return_type)
        return  elim_result_df, elim_stat_df
    else:          
        return result_dfpl, stat_df
        
    
    

In [ ]:
# Ver 3
def falsebreak_wDiv_test_ver2(dfpl,side, fwdCandle, div_indi = "MACD", falsebreak_bar = 0, 
                         ATR_surpass = 0, ATR_commpare_range = 0, div_signal = 'divSignal2MACD',
                         elim_cons_trade = True, return_type = pip_return_masking):
    '''
    side: 'buy' or 'sell'
    Test false break signal on divergence
    
    div_indi : indicator used for divergence signal.
    
    ATR_surpass: distance between high of break bar and currHigh as multipler of ATR
    
    falsebreak_bar: number of bars after false break considered to be signal
    
    fwdCandle: num of candles to add to Pivot before considered currHigh/Low
    
    div_signal: type of divergence signal: divSignalMACD, divSignal2MACD, divSignal3MACD, or 'No' to not use divergence vv..
    
    elim_cons_trade: eliminate consecutive trades, one trade per time
    
    return_type = pip_return_masking or log_return_masking
    '''
    dfpl['CurrLow'] = 0
    dfpl['CurrHigh'] = 0
    fwdCandle = fwdCandle
    for i in dfpl['pivot10bar'].index:
        if dfpl['pivot10bar'][i] == 1:
            dfpl['CurrLow'][i+fwdCandle:] = dfpl.low[i]
        elif dfpl['pivot10bar'][i] == 2:
            dfpl['CurrHigh'][i+fwdCandle:] = dfpl.high[i]
    dfpl[f'maxHi{fwdCandle}bar'] = dfpl.high.rolling(fwdCandle).max()
    dfpl[f'maxLo{fwdCandle}bar'] = dfpl.low.rolling(fwdCandle).min()
 
           
    # ----Sell side------------------
        
    if side.lower() == "sell":    
        
        false_break_masking_0bar =  (dfpl.high > dfpl.CurrHigh) & (
                                    dfpl.close < dfpl.CurrHigh) & (
                                    dfpl.high == dfpl[f'maxHi{fwdCandle}bar']) & (
                                    dfpl.UpperWick > ATR_commpare_range * dfpl['ATR']) & (
                                    # dfpl.UpperWick >dfpl.OpenCloseRange) & ( # adding this condition refine the result but too few trades
                                    # abs(dfpl.high - dfpl.low) >= ATR_commpare_range*dfpl['ATR']) & (
                                    # dfpl.close > dfpl.EMA20) & (
                                    # dfpl.open > dfpl.EMA20) & (
                                    # dfpl.close < dfpl.open) & (
                                    # abs(dfpl.high - dfpl.close) > abs(dfpl.close - dfpl.open)) & (

                                    
                                    dfpl.high.shift(periods = 1) < dfpl.CurrHigh) & (
                                    dfpl.HiLowRange.shift(periods = 1) < dfpl.HiLowRange) & (
                                    # dfpl.close.shift(periods = 1) < dfpl.CurrHigh) & (
                                    # dfpl.open.shift(periods = 1) < dfpl.CurrHigh) & (
            
                                    dfpl.high.shift(periods = 2) < dfpl.CurrHigh) & (
                                    dfpl.HiLowRange.shift(periods = 2) < dfpl.HiLowRange) & (
                                    # dfpl.close.shift(periods = 2) < dfpl.CurrHigh) & (
                                    # dfpl.open.shift(periods = 2) < dfpl.CurrHigh) & (
                                    # dfpl.close.shift(periods = 3) < dfpl.CurrHigh) & (
                                    # dfpl.close < dfpl.open.shift(periods = 1)) & (
            
                                    abs(dfpl.high-dfpl.CurrHigh) >= ATR_surpass * dfpl['ATR'])
                                    # abs(dfpl.high - dfpl.close) > abs(dfpl.open - dfpl.close))  & (
        
        false_break_masking_1bar =  (dfpl.open > dfpl.CurrHigh) & (
                                    dfpl.close < dfpl.CurrHigh) & (                                    
                                    dfpl.close < (dfpl.high.shift(periods = 1) - dfpl.low.shift(periods = 1))/2) & (
            
                                    dfpl.open.shift(periods = 1) <dfpl.CurrHigh) & (
                                    dfpl.close.shift(periods = 1) >dfpl.CurrHigh) & (
                                    dfpl.high.shift(periods = 1) == dfpl[f'maxHi{fwdCandle}bar']) & (
                                    
                                    dfpl.high.shift(periods = 2) < dfpl.CurrHigh) & (
                                    # dfpl.close.shift(periods = 2) < dfpl.CurrHigh) & (
                                    # dfpl.open.shift(periods = 2) < dfpl.CurrHigh) & (
            
                                    dfpl.high.shift(periods = 3) < dfpl.CurrHigh) & ( 
                                    # dfpl.close.shift(periods = 3) < dfpl.CurrHigh) & (
                                    # dfpl.open.shift(periods = 3) < dfpl.CurrHigh) 
                                    # dfpl.close.shift(periods = 4) < dfpl.CurrHigh)
                                    abs(dfpl.high.shift(periods = 1)-dfpl.CurrHigh) >= ATR_surpass * dfpl['ATR']) 

        
        if falsebreak_bar == 0:
            false_break_masking = false_break_masking_0bar
        elif falsebreak_bar == 1:
            false_break_masking = false_break_masking_1bar 
        elif falsebreak_bar == 100:
            false_break_masking = false_break_masking_0bar | false_break_masking_1bar
        else :
            warnings.warn("Warning : maximum falsebreak_bar is currrently <= 2 ")
            
        if div_signal.lower() == 'no':
            masking =  false_break_masking
        else:
            divSignal_masking  = dfpl[div_signal] == 1
            masking = divSignal_masking & false_break_masking        
               
        
        #  -------------reuslt----------------------
        
        result_dfpl = dfpl[masking]        
        stat_df = result_dfpl[return_type].describe().T
        stat_df['Sharpe ratio'] = stat_df['mean']/stat_df['std']
        stat_df['Count x Sharpe'] = stat_df['count']*stat_df['Sharpe ratio']
        
# ----Buy side-------------------------------------------------------

    elif side.lower() == 'buy':
           
        false_break_masking_0bar =  (dfpl.low < dfpl.CurrLow) & (
                                    dfpl.close > dfpl.CurrLow) & (
                                    dfpl.low == dfpl[f'maxLo{fwdCandle}bar']) & (
                                    # dfpl.LowerWick >dfpl.OpenCloseRange) & ( # adding this condition refine the result but too few trades
                                    dfpl.LowerWick > ATR_commpare_range * dfpl['ATR']) & (
                                    # abs(dfpl.high - dfpl.low) >= ATR_commpare_range*dfpl['ATR']) & (
            
                                    # dfpl.close < dfpl.EMA20) & ( 
                                    # dfpl.open < dfpl.EMA20) & ( 
                                    # dfpl.close > dfpl.open) & (                                    
                                    # abs(dfpl.low - dfpl.close) > abs(dfpl.close - dfpl.open)) & (
                                    # dfpl.close > dfpl.open) & (
                                    
                                    dfpl.low.shift(periods = 1) > dfpl.CurrLow) & (
                                    dfpl.HiLowRange.shift(periods = 1) < dfpl.HiLowRange) & (
                                    # dfpl.close.shift(periods = 1) > dfpl.CurrLow) & (
                                    # dfpl.open.shift(periods = 1) > dfpl.CurrLow) & (
            
                                    dfpl.low.shift(periods = 2) > dfpl.CurrLow) & (
                                    dfpl.HiLowRange.shift(periods = 2) < dfpl.HiLowRange) & (
                                    # dfpl.close.shift(periods = 2) > dfpl.CurrLow) & (
                                    # dfpl.open.shift(periods = 2) > dfpl.CurrLow) & (
                                    # dfpl.close.shift(periods = 3) > dfpl.CurrLow) & (
                                    # dfpl.close > dfpl.open.shift(periods = 1)) & (
            
                                    abs(dfpl.low - dfpl.CurrLow) >= ATR_surpass * dfpl['ATR']) 
                                    # abs(dfpl.low - dfpl.close) > abs(dfpl.open - dfpl.close))& (
                                    
                    
        false_break_masking_1bar = (dfpl.open < dfpl.CurrLow) & (
                                    dfpl.close > dfpl.CurrLow) & (
                                    dfpl.close > (dfpl.high.shift(periods = 1) - dfpl.low.shift(periods = 1))/2) & (
                                    # dfpl.close > dfpl.open.shift(periods = 1)) & (                      


                                    dfpl.open.shift(periods = 1) > dfpl.CurrLow) & (
                                    dfpl.close.shift(periods = 1) < dfpl.CurrLow) & (
                                    dfpl.low.shift(periods = 1) == dfpl[f'maxLo{fwdCandle}bar']) & (
            
                                    dfpl.low.shift(periods = 2) > dfpl.CurrLow) & (
                                    # dfpl.close.shift(periods = 2) > dfpl.CurrLow) & (
                                    # dfpl.open.shift(periods = 2) > dfpl.CurrLow) & (
                            
                                    dfpl.low.shift(periods = 3) > dfpl.CurrLow) & (
                                    # dfpl.close.shift(periods = 3) > dfpl.CurrLow) & (
                                    # dfpl.open.shift(periods = 3) > dfpl.CurrLow) & (
            
                                    # dfpl.close.shift(periods = 4) > dfpl.CurrLow) & (
                                    abs(dfpl.low.shift(periods = 1) - dfpl.CurrLow) >= ATR_surpass * dfpl['ATR'])
        
        if falsebreak_bar == 0:
            false_break_masking = false_break_masking_0bar
        elif falsebreak_bar == 1:
            false_break_masking = false_break_masking_1bar 
        elif falsebreak_bar == 100:
            false_break_masking = false_break_masking_0bar | false_break_masking_1bar
        else :
            warnings.warn("Warning : maximum falsebreak_bar is currrently <= 2 ")  
            
        if div_signal.lower() == "no":
            masking =  false_break_masking
        else:
            divSignal_masking  = dfpl[div_signal] == 2
            masking = divSignal_masking & false_break_masking   
        
        #-------------reuslt----------------------

        result_dfpl = dfpl[masking] # result_df of the test         
        stat_df = result_dfpl[return_type].describe().T
        stat_df['Sharpe ratio'] = stat_df['mean']/stat_df['std']
        stat_df['Count x Sharpe'] = stat_df['count']*stat_df['Sharpe ratio'] # stat df of the test
        
#         -------eliminate consecutive trades------------
        
    print ( f'result of {side} falsebreak with falsebreak bar = {falsebreak_bar},\n, ATR_Surpass = {ATR_surpass}, div_signal = {div_signal}, elim_cons_trade = {elim_cons_trade},return_type = {return_type} ')
    
    if elim_cons_trade:        
        elim_result_df, elim_stat_df = elimConsTrades(result_dfpl, elim_return_type = return_type)
        return  elim_result_df, elim_stat_df
    else:          
        return result_dfpl, stat_df

## Plot Fuctions

#### Plot function: ===> Plot_csPipReturn()

In [ ]:
def Plot_csPipReturn(df, side, spread, return_bar, figsize = (10,6)):
    '''
    df: resulf_dataframe which already have signal
    return_bar:  return after signal xx bar. Value = 1,3,10,20 or "All"
    side: 'buy' or 'sell'
    spread: spread to deduced from return, noting spread is set according to pip_return x multiplier (trading_test class)
    '''
    vnames = [name for name in globals() if globals()[name] is df]
    fig, ax = plt.subplots(figsize = figsize)
    if side == 'sell':
        if return_bar == "All":
            for rtBar in (1,3,5,10,15,20):
                df[f'PipReturn_{rtBar}bar_cumsum'] = (df[f'PipReturn_{rtBar}bar'] + spread).cumsum()
                ax.plot(df.index, -df[f'PipReturn_{rtBar}bar_cumsum'], label = f' return {rtBar} bar')
        else:    
            df[f'PipReturn_{return_bar}bar_cumsum'] = (df[f'PipReturn_{return_bar}bar'] + spread).cumsum()
            ax.plot(df.index, -df[f'PipReturn_{return_bar}bar_cumsum'])
        
    elif side == 'buy' or side == 'All':
        if return_bar == "All":
            for rtBar in (1,3,5,10,15,20):
                df[f'PipReturn_{rtBar}bar_cumsum'] = (df[f'PipReturn_{rtBar}bar'] - spread).cumsum()
                ax.plot(df.index, df[f'PipReturn_{rtBar}bar_cumsum'],label = f' return {rtBar} bar')
        else:
            df[f'PipReturn_{return_bar}bar_cumsum'] = (df[f'PipReturn_{return_bar}bar'] - spread).cumsum()
            ax.plot(df.index, df[f'PipReturn_{return_bar}bar_cumsum'])
    
    if side == "All":
        ax.set_title(label = f' {vnames} Culmulative pip return after {return_bar} bars for both side, spread = {spread}, trades = {len(df)}', loc = 'center')
    else:
        ax.set_title(label = f' {vnames} Culmulative pip return after {return_bar} bars for {side} side, spread = {spread}, trades = {len(df)}', loc = 'center')
    ax.set_xlabel("Trades")
    ax.set_ylabel('Culmulative return')
#     patch1 = mpatches.Patch(color="blue", label=" Return 1 bar")
#     patch2 = mpatches.Patch(color="green", label="Return 3 bar")
#     patch3 = mpatches.Patch(color="red", label="Return 10 bar")
#     patch4 = mpatches.Patch(color="orange", label="Return 20 bar")

    plt.legend()
    plt.show()
        
        


## Report Function

#### GetStatAll() ==> Get stats for both buy and sell side of a result_df

In [ ]:
def GetStatAll(resultAll_df):
    statAll_df = resultAll_df[pip_return_masking].describe().T
    statAll_df['Sharpe ratio'] = statAll_df['mean']/ statAll_df['std']
    statAll_df['Count x Sharpe'] = statAll_df['Sharpe ratio'] * statAll_df['count']
    return statAll_df

#### GetResultAll() ==> Get result_df for both side of trade

In [ ]:
def GetResultAll(resultSell_df, resultBuy_df):
    resultSell_df_copy = resultSell_df.copy()
    resultSell_df_copy[pip_return_masking]  = resultSell_df_copy[pip_return_masking]*-1
    resultAll_df = pd.concat([resultBuy_df, resultSell_df_copy])
    try:
        resultAll_df = resultAll_df.sort_values(by = "gmt time")
    except :
        try:
            resultAll_df = resultAll_df.sort_values(by = "local time")
        except:
            resultAll_df = resultAll_df.sort_values(by = "time")
    resultAll_df = resultAll_df.reset_index(drop = True)
    
    return resultAll_df

# Class TradingTest

### Main class

In [ ]:
class TradingTest:
        
    def __init__(self, df, pipRetMulti, MACD = True, fastMACD = 3,
                 slowMACD = 10,signalMACD = 16, RSI = True, lengthRSI = 30, lengthATR = 20):
        '''
        pipRetMulti: Multiplier of pip return to get pip value. 10 for XAUUSD:
                        100 for JPY-based pairs,
                        10.000 for USD,GBP,EUR,AUD,NZD,CHF based pairs
                        0.1 for BTC-USD
        '''
        self.df = df.copy()
        self.df.columns = list(map(str.lower,list(self.df.columns)))
        self.pipRetMulti = pipRetMulti
        self.compare_range = 0
        
        self.df['LogReturn_1bar'] = np.log(self.df['close'].shift(periods = -1 )/self.df['close'])
        self.df['LogReturn_3bar'] = np.log(self.df['close'].shift(periods = -3 )/self.df['close'])
        self.df['LogReturn_10bar'] = np.log(self.df['close'].shift(periods = -10 )/self.df['close'])
        self.df['LogReturn_20bar'] = np.log(self.df['close'].shift(periods = -20)/self.df['close'])
        self.log_return_masking = ['LogReturn_1bar','LogReturn_3bar','LogReturn_10bar','LogReturn_20bar']
        
        self.df['PipReturn_1bar'] = (self.df['close'].shift(periods = -1 ) - self.df['close']) * self.pipRetMulti
        self.df['PipReturn_3bar'] = (self.df['close'].shift(periods = -3 ) - self.df['close']) * self.pipRetMulti
        self.df['PipReturn_5bar'] = (self.df['close'].shift(periods = -5 ) - self.df['close']) * self.pipRetMulti
        self.df['PipReturn_10bar'] = (self.df['close'].shift(periods = -10 ) - self.df['close']) * self.pipRetMulti
        self.df['PipReturn_15bar'] = (self.df['close'].shift(periods = -15 ) - self.df['close']) * self.pipRetMulti
        self.df['PipReturn_20bar'] = (self.df['close'].shift(periods = -20 ) - self.df['close']) * self.pipRetMulti
        self.pip_return_masking = [['LogReturn_1bar','LogReturn_3bar','LogReturn_5bar','LogReturn_10bar',
                                    'LogReturn_15bar','LogReturn_20bar']]
        
        self.df['EMA20'] =  self.df.ta.ema(length = 20)
        self.df['HiLowRange'] =  abs(self.df.high - self.df.low)
        self.df['OpenCloseRange'] =  abs(self.df.open - self.df.close)
        self.df['UpperWick'] = self.df.high - self.df.close
        self.df['LowerWick'] = self.df.close - self.df.low

        if MACD:
            self.df['MACD'] = self.df.ta.sma(length = fastMACD) - self.df.ta.sma(length = slowMACD)
        if RSI:
            self.df['RSI'] = df.ta.rsi(length=lengthRSI)
        self.df['ATR'] = self.df.ta.atr(length = lengthATR)

#   ---------------------------------------------------------------------------------------------------------
            
    def graph_Sample(self, startID, endID, MACD = True, RSI = False, 
                     slider = False, legend = False, width = 1100, height = 600,
                     pp= True, RSIpp = False, MACDpp = True,
                     CurrHigh = True, CurrLow = True):
        
        dfpl = self.df.iloc[startID:endID]
        
        fig = make_subplots(rows=3, cols=1, subplot_titles=("Price", "MACD", "RSI"),
                            row_heights = [0.6 ,0.2, 0.2],
                            vertical_spacing= 0.02,
                            shared_xaxes = True)
        fig.append_trace(go.Candlestick(x=dfpl.index,
                                        open=dfpl['open'],
                                        high=dfpl['high'],
                                        low=dfpl['low'], 
                                        close=dfpl['close']), row=1, col=1)
        
        fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'],mode="markers",
                        marker=dict(size=5, color="MediumPurple"),name="pivot", row=1, col=1)
        
        if CurrHigh:
            fig.add_scatter(x=dfpl.index, y=dfpl['CurrHigh'],mode="markers",
                            marker=dict(size=3, color="MediumPurple"),
                            name="CurrPivotHigh", row=1, col=1)
        if CurrLow:
            fig.add_scatter(x=dfpl.index, y=dfpl['CurrLow'],mode="markers",
                            marker=dict(size=3, color="mediumvioletred"),name="CurrPivotLow", row=1, col=1)
        if MACD:
            fig.append_trace(go.Scatter(x=dfpl.index,y=dfpl['MACD']),row=2, col=1)
            fig.add_scatter(x=dfpl.index, y=dfpl['MACDpointpos'],mode="markers",
                            marker=dict(size=4, color="MediumPurple"),name="pivot", row=2, col=1)
        if RSI:
            fig.append_trace(go.Scatter(x=dfpl.index,y=dfpl['RSI']),row=3, col=1)
            fig.add_scatter(x=dfpl.index, y=dfpl['RSIpointpos'], mode="markers",marker=dict(size=4, color="MediumPurple"),
                                name="pivot", row=3, col=1)
          
        fig.update_layout(xaxis_rangeslider_visible = slider, showlegend = legend, height = height, width = width, margin=dict(l=20,r=20,b=20,t=20,pad=2))
        return fig    
    
#   ---------------------------------------------------------------------------------------------------------

    def gen_pivot(self, startRange, endRange , MACD = True, RSI = False, pp= True, RSIpp = False, MACDpp = True):
        '''
        rangeStart: number of previous bars to compare
        rangeEnd: number of subseqence bars to compare
        
        return a dataframe with pivot point of price, macd, rsi columns
        
        '''
        self.compare_range = startRange
        
        self.df['pivot'] = self.df.apply(lambda x: pivotid(self.df, x.name, startRange, endRange), axis=1)
        if MACD: 
            self.df['MACDpivot'] = self.df.apply(lambda x: MACDpivotid(self.df, x.name, startRange, endRange), axis=1)
        if RSI:
            self.df['RSIpivot'] = self.df.apply(lambda x: RSIpivotid(self.df, x.name, startRange, endRange), axis=1)
            
        if pp:
            self.df['pointpos'] =  self.df.apply(lambda row: pointpos(row), axis=1)
        if RSIpp:
            self.df['RSIpointpos'] =  self.df.apply(lambda row: RSIpointpos(row), axis=1)
        if MACDpp:
            self.df['MACDpointpos'] =  self.df.apply(lambda row: MACDpointpos(row), axis=1)

        print (f'pivot point generated with comparative range of( {startRange}, {endRange} ) candle')
    
#   ---------------------------------------------------------------------------------------------------------

    def gen_LrSlope(self,nbackcandles):
        self.df[f'LrSlope_{nbackcandles}bar'] = self.df.apply(lambda row: slopeGenerate(self.df, row, nbackcandles), axis=1)
        
        
#   ---------------------------------------------------------------------------------------------------------


    def gen_divSignal1(self, nbackcandles, MACD = True, RSI = False):
        if MACD:
            self.df['divSignalMACD'] = self.df.apply(lambda row: divsignalMACD(self.df,row, nbackcandles), axis=1)
        if RSI:
            self.df['divSignalRSI'] = self.df.apply(lambda row: divsignalRSI(self.df, row, nbackcandles), axis=1)
        return self.df
    
#   ---------------------------------------------------------------------------------------------------------

    
    def gen_divSignal2(self, nbackcandles, startRange = 10, MACD = True, RSI = False):
        # self.gen_pivot(startRange = startRange, endRange = startRange, MACD = MACD, MACDpp = MACD, RSI = RSI,RSIpp = RSI)        
        startRange = self.compare_range
        
        if MACD:
            self.df['divSignal2MACD'] = self.df.apply(lambda row: divsignal2MACD(self.df,row,nbackcandles,startRange), axis=1)
        if RSI:
            self.df['divSignal2RSI'] = self.df.apply(lambda row: divsignal2RSI(self.df,row,nbackcandles,startRange), axis=1)
        print(f'divergence signal 2 generated with comparative range of( {startRange}, {startRange} ) candle') 

#   ---------------------------------------------------------------------------------------------------------
    
    def gen_divSignal3(self, nbackcandles, startRange = 10, thresholdRSI_up = 0,thresholdRSI_down = 100):
        
        startRange = self.compare_range        
        self.df['divSignal3MACD'] = self.df.apply(lambda row: divsignal3MACD(self.df, row, nbackcandles, startRange, thresholdRSI_up,thresholdRSI_down), axis=1)
        
        print(f'divergence signal 3 generated with comparative range of( {startRange}, {startRange} ) candle') 
    
#   ---------------------------------------------------------------------------------------------------------

      
    
#   ---------------------------------------------------------------------------------------------------------

    def test_falseBreakSig2MACD(self,falsebreak_bar= 0,
                                ATR_surpass = 0,
                                ATR_commpare_range = 0,
                                fwdCandle = 0,
                                elim_cons_trade = True,
                                div_signal = 'divSignal2MACD'):
        
        fwdCandle = self.compare_range
        self.result_Sell, self.stats_Sell = falsebreak_wDiv_test(self.df, side = 'sell',
                                                                falsebreak_bar = falsebreak_bar,
                                                                ATR_surpass = ATR_surpass,
                                                                ATR_commpare_range = ATR_commpare_range,
                                                                fwdCandle = fwdCandle,
                                                                elim_cons_trade = elim_cons_trade,
                                                                div_signal = div_signal
                                                                )
        self.result_Buy, self.stats_Buy = falsebreak_wDiv_test(self.df,side = 'buy',
                                                                falsebreak_bar = falsebreak_bar,
                                                                ATR_surpass = ATR_surpass,
                                                                ATR_commpare_range = ATR_commpare_range,
                                                                fwdCandle = fwdCandle,
                                                                elim_cons_trade = elim_cons_trade,
                                                                div_signal = div_signal
                                                                )      
        self.result_All = GetResultAll(self.result_Sell,self.result_Buy)
        self.stats_All = GetStatAll(self.result_All)
        
        print('Sell stats')
        display(self.stats_Sell)
        
        print('Buy Stats')
        display(self.stats_Buy)
        
        print('All Stats')
        display(self.stats_All)
        
        display(self.plot_PipReturn())
#   ---------------------------------------------------------------------------------------------------------
 
    def plot_PipReturn(self, side = "All", spread = 2, return_bar = 'All'):
        '''
        side : 'All' or 'sell' or 'buy'
        spread : int 
        return_bar : int or 'All'
        
        '''
        if side == 'All':
            Plot_csPipReturn(self.result_All, side = side, spread = spread, return_bar = return_bar, figsize=(10, 6))
        elif side == 'sell':
            Plot_csPipReturn(self.result_Sell, side = side, spread = spread, return_bar = return_bar, figsize=(10, 6))
        elif side == 'buy':
            Plot_csPipReturn(self.result_Buy, side = side, spread = spread, return_bar = return_bar, figsize=(10, 6))
        



# Load data and Test

In [1]:
%pwd

'C:\\Users\\AMD G6\\Trading-test'

In [2]:
%store

Stored variables and their in-db values:


In [ ]:
%store -r EURAUD_15M_0822_0223_TDVIEW

In [ ]:
EURAUD_15M_0822_0223_TDVIEW_test = EURAUD_15M_0822_0223_TDVIEW[0]

In [ ]:
EURAUD_15M_0822_0223_TDVIEW_test.test_falseBreakSig2MACD(ATR_surpass=0.3,ATR_commpare_range = 0.3)

In [ ]:
%store -r EURAUD15M_0722_1222_Dukas               

In [ ]:
EURAUD15M_0722_1222_Dukas_test = EURAUD15M_0722_1222_Dukas[0]

In [ ]:
ATR_surpass = 0.8
selldf, statselldf = falsebreak_wDiv_test(EURAUD15M_0722_1222_Dukas_test.df,side = 'sEll',fwdCandle=10, ATR_surpass= ATR_surpass, elim_cons_trade=True, div_signal='NO')
buydf, statbuydf = falsebreak_wDiv_test(EURAUD15M_0722_1222_Dukas_test.df,side = 'bUy',fwdCandle=10, ATR_surpass= ATR_surpass, elim_cons_trade=True, div_signal='NO')
allsresulfdf = GetResultAll(selldf,buydf)
statall = GetStatAll(allsresulfdf)
display(statall)
Plot_csPipReturn(allsresulfdf,side = 'All',spread = 2,return_bar='All')

In [ ]:
%store -r EURAUD15M_0122_0622_list                

In [ ]:
EURAUD15M_0122_0622_test = EURAUD15M_0122_0622_list[1]

In [ ]:
ATR_surpass = 0.8
selldf, statselldf = falsebreak_wDiv_test(EURAUD15M_0122_0622_test.df,side = 'sEll',fwdCandle=10, ATR_surpass= ATR_surpass, elim_cons_trade=True, div_signal='NO')
buydf, statbuydf = falsebreak_wDiv_test(EURAUD15M_0122_0622_test.df,side = 'bUy',fwdCandle=10, ATR_surpass= ATR_surpass, elim_cons_trade=True, div_signal='NO')
allsresulfdf = GetResultAll(selldf,buydf)
statall = GetStatAll(allsresulfdf)
display(statall)
Plot_csPipReturn(allsresulfdf,side = 'All',spread = 2,return_bar='All')

In [ ]:
%store -r saved_list                              

In [ ]:
saved_list

## VN30F1M

In [ ]:
VM30F1_15M_2023 = pd.read_csv("D:Data/Stock/VN/Future/TDView_HNX_DLY_VN301!, 15, 20_23.csv")

In [ ]:
VM30F1_15M_2023_test = TradingTest(VM30F1_15M_2023,pipRetMulti=1)

In [ ]:
VM30F1_15M_2023_test

In [ ]:
compare_range = 10

In [ ]:
VM30F1_15M_2023_test.gen_pivot(10,10)

In [ ]:
datetime.datetime.today()

In [ ]:
saved_list = [divsignal2MACD,VM30F1_15M_2023_test]
%store saved_list

In [ ]:
%store -r saved_list

In [ ]:
a = saved_list[0]

In [ ]:
VM30F1_15M_2023_test = saved_list[1]

In [ ]:
VM30F1_15M_2023_test.graph_Sample(100,200,CurrHigh = False, CurrLow= False)

In [ ]:
VM30F1_15M_2023_test.gen_divSignal2(100,10)

In [ ]:
VM30F1_15M_2023_test.compare_range

In [ ]:
VM30F1_15M_2023_test.test_falseBreakSig2MACD(ATR_surpass = 0.2)

In [ ]:
VM30F1_15M_2023_test.plot_PipReturn('All')

In [ ]:
VM30F1_15M_2023_test.result_Sell

In [ ]:
VM30F1_15M_2023_test.graph_Sample(3566,3688)

## XAUUSD

###  5 min

#### Trading view data- OANDA

In [ ]:
XAU5M_1222_0223 = pd.read_csv("D:Data/Forex/XAUUSD/OANDA_XAUUSD, 5, 1222_0223 .csv")
XAU5M_1222_0223_test = TradingTest(XAU5M_1222_0223, pipRetMulti= 10)

In [ ]:
compare_range = 10
XAU5M_1222_0223_test.gen_pivot(compare_range,compare_range)

In [ ]:
XAU5M_1222_0223_test.gen_divSignal2(compare_range*10, compare_range)

In [ ]:
XAU5M_1222_0223_test.test_falseBreakSig2MACD(ATR_surpass=0.3)

In [ ]:
XAU5M_1222_0223_test.result_Sell

In [ ]:
a = 363
XAU5M_1222_0223_test.graph_Sample(a-100,a+20)

In [ ]:
startID = 10732
endID = 10843
dfpl = XAU5M_1222_0223_test.df[startID:endID]
width = 1100
height = 600
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("Price", "MACD", "RSI"),
                    row_heights=[0.6, 0.2, 0.2], vertical_spacing= 0.02,shared_xaxes = True )
fig.append_trace(go.Candlestick(x=dfpl.index,
        open=dfpl['open'],
        high=dfpl['high'],
        low=dfpl['low'],
        close=dfpl['close']), row=1, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),name="pivot", row=1, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['CurrHigh'],mode="markers",
                    marker=dict(size=3, color="MediumPurple"),name="CurrPivotHigh", row=1, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['CurrLow'],mode="markers",
                    marker=dict(size=3, color="mediumvioletred"),name="CurrPivotLow", row=1, col=1)
fig.append_trace(go.Scatter(x=dfpl.index,y=dfpl['MACD']),row=2, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['MACDpointpos'],mode="markers",
                    marker=dict(size=4, color="MediumPurple"),name="pivot", row=2, col=1)

fig.append_trace(go.Scatter(x=dfpl.index,y=dfpl['RSI']),row=3, col=1)
# fig.add_scatter(x=dfpl.index, y=dfpl['RSIpointpos'], mode="markers",marker=dict(size=4, color="MediumPurple"),
#                         name="pivot", row=3, col=1)
fig.update_layout(xaxis_rangeslider_visible= False, showlegend=False, height = height, width = width, margin=dict(l=50,r=50,b=0,t=0,pad=0))

#### DUKASCOPY_XAUUSD_5M_BID_01.09.2022-14.01.2023

In [ ]:
df = pd.read_csv("D:Data/Forex/XAUUSD/OANDA_XAUUSD, 5, 1222_0223 .csv")
tradingTest_xau5min = TradingTest(df)
tradingTest_xau5min.gen_pivot(5,5)

In [ ]:
tradingTest_xau5min.df.columns

In [ ]:
tradingTest_xau5min.gen_divSignal2(30, RSI = False)

In [ ]:
tradingTest_xau5min.df['divSignal2MACD'].value_counts()

In [ ]:
sell_result_dfpl, sell_stat_df = falsebreak_wDiv_test(tradingTest_xau5min.df, side = 'sell', elim_cons_trade = False, ATR_surpass= 0.1)
sell_stat_df

In [ ]:
sell_result_dfpl, sell_stat_df = falsebreak_wDiv_test(tradingTest_xau5min.df, side = 'sell', elim_cons_trade = True)
sell_stat_df

In [ ]:
buy_result_dfpl, buy_stat_df = falsebreak_wDiv_test(tradingTest_xau5min.df, side = 'buy', elim_cons_trade = True, ATR_surpass= 0.1)
buy_stat_df

#### DUKASCOPY_XAUUSD_Candlestick_5_M_BID_01.05.2022-01.09.2022

In [ ]:
print(os.listdir("D:/5.Data center/Forex/XAUUSD/"))

In [ ]:
df = pd.read_csv("D:/5.Data center/Forex/XAUUSD/DUKASCOPY_XAUUSD_Candlestick_5_M_BID_01.05.2022-01.09.2022.csv")
df.columns = ['Gmt time', 'open', 'high', 'low', 'close', 'Volume']
tradingTest_xau5min = TradingTest(df)
tradingTest_xau5min.df

In [ ]:
tradingTest_xau5min.gen_pivot(5,5)

In [ ]:
tradingTest_xau5min.gen_divSignal2(30)

In [ ]:
tradingTest_xau5min.df.divSignal2MACD.value_counts()

##### Sell

In [ ]:
sell_result_dfpl, sell_stat_df = falsebreak_wDiv_test(tradingTest_xau5min.df, side = 'sell', elim_cons_trade = False , falsebreak_bar=0, ATR_surpass=0.1)
sell_stat_df

##### Buy

In [ ]:
buy_result_dfpl, buy_stat_df = falsebreak_wDiv_test(tradingTest_xau5min.df, side = 'buy', elim_cons_trade = True , falsebreak_bar=1, ATR_surpass=0.1)
buy_stat_df

#### DUKAS_XAUUSD_Candlestick_5_M_BID_01.01.2021-01.05.2021

In [ ]:
df = pd.read_csv("D:/Data/Forex/XAUUSD/DUKAS_XAUUSD_Candlestick_5_M_BID_01.01.2021-01.05.2021.csv")
df.columns = ['Gmt time', 'open', 'high', 'low', 'close', 'Volume']
tradingTest_xau5min = TradingTest(df, pipRetMulti = 10)
tradingTest_xau5min.df

In [ ]:
tradingTest_xau5min.gen_pivot(5,5)

In [ ]:
tradingTest_xau5min.gen_divSignal2(30)

In [ ]:
XAU_sell_result_dfpl, XAU_sell_stat_df = falsebreak_wDiv_test(tradingTest_xau5min.df, side = 'sell', elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass=0.05)
XAU_sell_stat_df

In [ ]:
XAU_sell_result_dfpl['PipReturn_10bar_cumsum'] = (XAU_sell_result_dfpl['PipReturn_10bar']+2).cumsum()
plt.plot(XAU_sell_result_dfpl.index, -XAU_sell_result_dfpl['PipReturn_10bar_cumsum'])

##### buy

In [ ]:
XAU_buy_result_dfpl, XAU_buy_stat_df = falsebreak_wDiv_test(tradingTest_xau5min.df, side = 'buy', elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass= 0.05)
XAU_buy_stat_df

In [ ]:
XAU_buy_result_dfpl['PipReturn_10bar_cumsum'] = (XAU_buy_result_dfpl['PipReturn_10bar']-2).cumsum()
plt.plot(XAU_buy_result_dfpl.index, XAU_buy_result_dfpl['PipReturn_10bar_cumsum'])

### 15 Min

In [ ]:
XAU15M_20_23 = pd.read_csv("D:/Data/Forex/XAUUSD/DUKAS_XAUUSD_Candlestick_15_M_BID_01.01.2020-01.01.2023.csv")

In [ ]:
XAU15M_20_23['Gmt time'] = pd.to_datetime(XAU15M_20_23['Gmt time'])
XAU15M_20_23 = XAU15M_20_23.sort_values(by = 'Gmt time')

In [ ]:
XAU_15m2020 = XAU15M_20_23[XAU15M_20_23['Gmt time'] < '2021-1-1']
XAU_15m2021 = XAU15M_20_23[(XAU15M_20_23['Gmt time'] > '2021-1-1') &  (XAU15M_20_23['Gmt time'] < '2022-1-1')]
XAU_15m2022 = XAU15M_20_23[XAU15M_20_23['Gmt time'] >= '2022-1-1']

In [ ]:
XAU15M_17_20 = pd.read_csv("D:/Data/Forex/XAUUSD/XAUUSD_Candlestick_15_M_BID_01.01.2017-01.01.2020.csv")
XAU15M_17_20['Gmt time'] = pd.to_datetime(XAU15M_17_20['Gmt time'])
XAU15M_17_20 = XAU15M_17_20.sort_values(by = 'Gmt time')

In [ ]:
XAU_15m2017 = XAU15M_17_20[XAU15M_17_20['Gmt time'] < '2018-1-1']
XAU_15m2018 = XAU15M_17_20[(XAU15M_17_20['Gmt time'] > '2017-12-30') &  (XAU15M_17_20['Gmt time'] < '2019-1-1')]
XAU_15m2019 = XAU15M_17_20[XAU15M_17_20['Gmt time'] >= '2018-12-30']

#### 2017

In [ ]:
XAU_15m2017_test = TradingTest(XAU_15m2017.reset_index().drop(columns = ['index']), pipRetMulti = 10)

In [ ]:
XAU_15m2017_test.gen_pivot(5,5)


In [ ]:
XAU_15m2017_test.gen_divSignal2(30,5)

In [ ]:
XAU_15m2017_test.df.divSignal2MACD.value_counts()

In [ ]:
XAU_15m2017_resultS, XAU_15m2017_statS = falsebreak_wDiv_test(XAU_15m2017_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)
XAU_15m2017_resultB, XAU_15m2017_statB = falsebreak_wDiv_test(XAU_15m2017_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)
XAU_15m2017_resultAll = GetResultAll(XAU_15m2017_resultS,XAU_15m2017_resultB)
XAU_15m2017_statAll = GetStatAll(XAU_15m2017_resultAll)

In [ ]:
Plot_csPipReturn(XAU_15m2017_resultAll,side = 'All',spread = 2, return_bar='All')

In [ ]:
XAU_15m2017_statAll

In [ ]:
Plot_csPipReturn(XAU_15m2017_resultS, side = 'sell', spread = 2, return_bar = 'All')
Plot_csPipReturn(XAU_15m2017_resultB, side = 'buy', spread = 2, return_bar = 'All')

In [ ]:
dfpl = XAU_15m2017_resultAll
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl.index,y=dfpl['PipReturn_10bar_cumsum']), row = 1, col = 1)    
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

In [ ]:
XAU_15m2017_resultAll.loc[70]

In [ ]:
XAU_15m2017_test.df.loc[11891]

In [ ]:
XAU_15m2017_test.graph_init_Sample(11800, 11920,RSI=False)

#### 2018

In [ ]:
XAU_15m2018_test = TradingTest(XAU_15m2018.reset_index(drop = True), pipRetMulti = 10)

In [ ]:
XAU_15m2018_test.gen_pivot(5,5)
XAU_15m2018_test.gen_divSignal2(30,5)

In [ ]:
XAU_15m2018_test.df.divSignal2MACD.value_counts()

In [ ]:
XAU_15m2018_resultS, XAU_15m2018_statS = falsebreak_wDiv_test(XAU_15m2018_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass= 0.0)
XAU_15m2018_resultB, XAU_15m2018_statB = falsebreak_wDiv_test(XAU_15m2018_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass= 0.0)
XAU_15m2018_resultAll = GetResultAll(XAU_15m2018_resultS,XAU_15m2018_resultB)
XAU_15m2018_statAll = GetStatAll(XAU_15m2018_resultAll)

In [ ]:
Plot_csPipReturn(XAU_15m2018_resultAll,side = 'All',spread = 2, return_bar='All')

In [ ]:
XAU_15m2018_statAll

In [ ]:
dfpl = XAU_15m2018_resultAll
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl['gmt time'],y=dfpl['PipReturn_3bar_cumsum']), row = 1, col = 1)
# fig.add_scatter(x = GJ1H_18_23_resultAll.iloc[138:]['gmt time'], y= GJ1H_18_23_resultAll.iloc[138:]['PipReturn_20bar_cumsum'],mode="markers",
#                             marker=dict(size=4, color="MediumPurple"),name="pivot", row=2, col=1)
       
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

#### 2019

In [ ]:
XAU_15m2019_test = TradingTest(XAU_15m2019.reset_index().drop(columns = ['index']), pipRetMulti = 10)

In [ ]:
XAU_15m2019_test.gen_pivot(5,5)
XAU_15m2019_test.gen_divSignal2(30)

In [ ]:
XAU_15m2019_test.df.divSignal2MACD.value_counts()

In [ ]:
XAU_15m2019_resultS, XAU_15m2019_statS = falsebreak_wDiv_test(XAU_15m2019_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0)
XAU_15m2019_resultB, XAU_15m2019_statB = falsebreak_wDiv_test(XAU_15m2019_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0)
XAU_15m2019_resultAll = GetResultAll(XAU_15m2019_resultS,XAU_15m2019_resultB)
XAU_15m2019_statAll = GetStatAll(XAU_15m2019_resultAll)

In [ ]:
Plot_csPipReturn(XAU_15m2019_resultAll,'All',2,'All')

In [ ]:
dfpl = XAU_15m2019_resultAll
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl['gmt time'],y=dfpl['PipReturn_3bar_cumsum']), row = 1, col = 1)    
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

In [ ]:
XAU_15m2019_statAll

In [ ]:
Plot_csPipReturn(XAU_15m2019_resultS, side = 'sell', spread = 2, return_bar = 'All')
Plot_csPipReturn(XAU_15m2019_resultB, side = 'buy', spread = 2, return_bar = 'All')

In [ ]:
XAU_15m2019_statS

In [ ]:
XAU_15m2019_statB

#### 2020

In [ ]:
XAU_15m2020_test = TradingTest(XAU_15m2020.reset_index().drop(columns = ['index']), pipRetMulti = 10)

In [ ]:
XAU_15m2020_test.gen_pivot(5,5)
XAU_15m2020_test.gen_divSignal2(30)

In [ ]:
XAU_15m2020_test.df.divSignal2MACD.value_counts()

In [ ]:
XAU_15m2020_resultS, XAU_15m2020_statS = falsebreak_wDiv_test(XAU_15m2020_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0)
XAU_15m2020_resultB, XAU_15m2020_statB = falsebreak_wDiv_test(XAU_15m2020_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0)
XAU_15m2020_resultAll = GetResultAll(XAU_15m2020_resultS,XAU_15m2020_resultB)
XAU_15m2020_statAll = GetStatAll(XAU_15m2020_resultAll)

In [ ]:
Plot_csPipReturn(XAU_15m2020_resultAll,'All',2,'All')

In [ ]:
XAU_15m2020_statAll

In [ ]:
dfpl = XAU_15m2020_resultAll
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl['gmt time'],y=dfpl['PipReturn_20bar_cumsum']), row = 1, col = 1)    
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

In [ ]:
XAU_15m2020_resultAll[XAU_15m2020_resultAll['gmt time'] >= '2020-2-11']

In [ ]:
XAU_15m2020[XAU_15m2020['Gmt time'] >= '2020-2-12 20:30:00']

In [ ]:
XAU_15m2020_test.graph_init_Sample(2800,2850,RSI = False,RSIpp=False,height=2000)

In [ ]:
Plot_csPipReturn(XAU_15m2020_resultS, side = 'sell', spread = 2, return_bar = 'All')
Plot_csPipReturn(XAU_15m2020_resultB, side = 'buy', spread = 2, return_bar = 'All')

#### 2021

In [ ]:
XAU_15m2021_test = TradingTest(XAU_15m2021.reset_index().drop(columns = ['index']), pipRetMulti = 10)

In [ ]:
XAU_15m2021_test.gen_pivot(5,5)
XAU_15m2021_test.gen_divSignal2(30)

In [ ]:
XAU_15m2021_test.df.divSignal2MACD.value_counts()

In [ ]:
XAU_15m2021_resultS, XAU_15m2021_statS = falsebreak_wDiv_test(XAU_15m2021_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0)
XAU_15m2021_resultB, XAU_15m2021_statB = falsebreak_wDiv_test(XAU_15m2021_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0)
XAU_15m2021_resultS_copy = XAU_15m2021_resultS.copy()
XAU_15m2021_resultS_copy[pip_return_masking]  = XAU_15m2021_resultS_copy[pip_return_masking]*-1
XAU_15m2021_resultAll = pd.concat([XAU_15m2021_resultB, XAU_15m2021_resultS_copy])
XAU_15m2021_resultAll = XAU_15m2021_resultAll.reset_index().drop(columns = ['level_0'])
Plot_csPipReturn(XAU_15m2021_resultAll, 'buy', 2, "All")


In [ ]:
XAU_15m2021_statAll = XAU_15m2021_resultAll[pip_return_masking].describe().T
XAU_15m2021_statAll['Sharpe ratio'] = XAU_15m2021_statAll['mean']/ XAU_15m2021_statAll['std']
XAU_15m2021_statAll['Count x Sharpe'] = XAU_15m2021_statAll['Sharpe ratio'] * XAU_15m2021_statAll['count']
XAU_15m2021_statAll

In [ ]:
XAU_15m2021_statS

In [ ]:
XAU_15m2021_statB

In [ ]:
Plot_csPipReturn(XAU_15m2021_resultS, 'sell', 2, 'All')

In [ ]:
Plot_csPipReturn(XAU_15m2021_resultB, 'buy', 2, 'All')

#### 2022

##### Dukas Data

In [ ]:
XAU_15m2022 = pd.read_csv('D:Data/Forex/XAUUSD/DUKAS_XAUUSD_Candlestick_15_M_BID_01.01.2022-14.01.2023.csv')
XAU_15m2022['Gmt time'] = pd.to_datetime(XAU_15m2022['Gmt time'], dayfirst=True)
XAU_15m2022 = XAU_15m2022.sort_values(by = 'Gmt time')
XAU_15m2022 = XAU_15m2022.reset_index(drop = True)

In [ ]:
XAU_15m2022

In [ ]:
XAU_15m2022_test = TradingTest(XAU_15m2022, pipRetMulti = 10)

In [ ]:
compare_range = 10

In [ ]:
XAU_15m2022_test.gen_pivot(compare_range,compare_range)


In [ ]:
XAU_15m2022_test.gen_divSignal2(compare_range*10, compare_range)

In [ ]:
XAU_15m2022_Dukas_TdTest_saved = [XAU_15m2022_test]

In [ ]:
%store XAU_15m2022_Dukas_TdTest_saved
exit

In [ ]:
XAU_15m2022_test.df.divSignal2MACD.value_counts()

In [ ]:
XAU_15m2022_test.test_falseBreakSig2MACD(ATR_surpass = 0.3, div_signal = 'no',ATR_commpare_range = 1)

In [ ]:
XAU_15m2022_test.test_falseBreakSig2MACD(ATR_surpass=0.3, falsebreak_bar = 0)

In [ ]:
XAU_15m2022_test.result_Sell

In [ ]:
a = 15774
XAU_15m2022_test.graph_Sample(a-100,a+20)

In [ ]:
XAU_15m2022_test.df.loc[15769]

In [ ]:
XAU_15m2022_test.df[XAU_15m2022_test.df['gmt time'] == datetime.datetime(2022,8,31,23,30)]

In [ ]:
XAU_15m2022_test.df['gmt time']

In [ ]:
XAU_15m2022_test.result_Buy

In [ ]:
a = 15531
XAU_15m2022_test.graph_Sample(a-100,a+20)

#### 2022 - 2023 Oanda - Tradingview data

In [ ]:
print(os.listdir("D:/Data/Forex/XAUUSD/"))

In [ ]:
XAU15M_121322_012723 = pd.read_csv("D:/Data/Forex/XAUUSD/OANDA_XAUUSD, 15, 0822_0223.csv")
XAU15M_121322_012723_test = TradingTest(XAU15M_121322_012723,pipRetMulti=10)

In [ ]:
compare_range = 10

In [ ]:
XAU15M_121322_012723_test.gen_pivot(compare_range,compare_range)

In [ ]:
XAU15M_121322_012723_test.gen_divSignal2(100)

In [ ]:
XAU15M_121322_012723_test.test_falseBreakSig2MACD(ATR_surpass=0)

In [ ]:
XAU15M_121322_012723_test.df.time = pd.to_datetime(XAU15M_121322_012723_test.df.time)

In [ ]:
XAU15M_121322_012723_test.df[XAU15M_121322_012723_test.df.time.dt.date == datetime.date(2022,11,16)]

In [ ]:
XAU15M_121322_012723_test.df.loc[7097:7100]

In [ ]:
a = 7097

In [ ]:
XAU15M_121322_012723_test.graph_Sample(a-100, a+20)

In [ ]:
XAU15M_121322_012723_test.df.time

In [ ]:
a = 592
XAU15M_121322_012723_test.graph_Sample(a -100, a+20)

In [ ]:
XAU15M_0822_0223_TDVIEW_test_saved = [XAU15M_121322_012723_test]

In [ ]:
%store XAU15M_0822_0223_TDVIEW_test_saved
exit

In [ ]:
XAU15M_121322_012723_test.result_Buy

## GBPJPY

### 15 Min

#### Tradng View Data

In [ ]:
GJ15M_0822_0223 = pd.read_csv("D:/Data/Forex/GBPJPY/OANDA_GBPJPY, 15, 0922_0223.csv")

In [ ]:
GJ15M_0822_0223_test = TradingTest(GJ15M_0822_0223,pipRetMulti=100)

In [ ]:
GJ15M_0822_0223_test.gen_pivot(5,5)

In [ ]:
GJ15M_0822_0223_test.gen_divSignal2(50)

In [ ]:
GJ15M_0822_0223_test.df['pivot10bar'] = GJ15M_0822_0223_test.df.apply(lambda row: pivotid(GJ15M_0822_0223_test.df, row.name, 10, 10), axis=1)

In [ ]:
(GJ15M_0822_0223_test.df['pivot10bar']).value_counts()

In [ ]:
GJ15M_0822_0223_test.df.divSignal2MACD.value_counts()

In [ ]:
sellresult, sellstat = falsebreak_wDiv_test_ver2(GJ15M_0822_0223_test.df,side = 'sell',fwdCandle =10,ATR_surpass=0.3)
buyresult, buystat = falsebreak_wDiv_test_ver2(GJ15M_0822_0223_test.df,side = 'buy',fwdCandle =10,ATR_surpass=0.3)


In [ ]:
sellstat

In [ ]:
sellresult

In [ ]:
a = 7621
GJ15M_0822_0223_test.graph_Sample(a-100, a+20)

In [ ]:
self.df['pivot'] = self.df.apply(lambda x: pivotid(self.df, x.name, startRange, endRange), axis=1)

In [ ]:
GJ15M_0822_0223_test_tdview_saved = [GJ15M_0822_0223_test]

In [ ]:
%store GJ15M_0822_0223_test_tdview_saved
exit

In [ ]:
GJ15M_0822_0223_test.test_falseBreakSig2MACD(ATR_surpass=0.5, div_signal = 'no',ATR_commpare_range=1)

In [ ]:
GJ15M_0822_0223_test.df

In [ ]:
GJ15M_0822_0223_test.df.time = GJ15M_0822_0223_test.df.time.astype('datetime64')

In [ ]:
GJ15M_0822_0223_test.df.time[GJ15M_0822_0223_test.df.time == datetime.datetime(2022,11,16,11,30)]

In [ ]:
GJ15M_0822_0223_test.df.loc[5080]

In [ ]:
a = 5072   
GJ15M_0822_0223_test.graph_Sample(a -120, a+20)

In [ ]:
GJ15M_0822_0223_test.test_falseBreakSig2MACD(ATR_surpass=0.3)

In [ ]:
GJ15M_0822_0223_test.result_Sell

In [ ]:
a = 3223
GJ15M_0822_0223_test.graph_Sample(a-100, a+20)

In [ ]:
GJ15M_0822_0223_statB

In [ ]:
GJ15M_0822_0223_resultB.tail(5)

In [ ]:
GJ15M_0822_0223_test.graph_init_Sample(5829,5939,height=1500)

In [ ]:
GJ15M_0822_0223_resultAll = GetResultAll(GJ15M_0822_0223_resultS,GJ15M_0822_0223_resultB)
GJ15M_0822_0223_stattAll = GetStatAll(GJ15M_0822_0223_resultAll)
GJ15M_0822_0223_stattAll

In [ ]:
Plot_csPipReturn(GJ15M_0822_0223_resultAll,'All',2,'All')

In [ ]:
print(os.listdir("D:/Data/Forex/GBPJPY/"))

In [ ]:
GJ15M_20_23 = pd.read_csv("D:/Data/Forex/GBPJPY/DUKAS_GBPJPY_Candlestick_15_M_BID_01.10.2020-01.01.2023.csv")

In [ ]:
GJ15M_20_23['Gmt time'] = pd.to_datetime(GJ15M_20_23['Gmt time'])

In [ ]:
GJ15M_2020 = GJ15M_20_23[GJ15M_20_23['Gmt time'] < '2021-1-1']
GJ15M_2021 = GJ15M_20_23[(GJ15M_20_23['Gmt time'] > '2021-1-1') &  (GJ15M_20_23['Gmt time'] < '2022-1-1')]
GJ15M_2022 = GJ15M_20_23[GJ15M_20_23['Gmt time'] >= '2022-1-1']

In [ ]:
GJ15M_2021[GJ15M_2021['Gmt time'] == '2021-11-18']

In [ ]:
GJ15M_20_23_statS

#### DUKASCOPY DATA

##### 2022-2023

In [ ]:
GJ15M_0122_0223 = pd.read_csv('D:Data/Forex/GBPJPY/DUKAS_GBPJPY_Candlestick_15_M_BID_01.01.2022-09.02.2023.csv')
GJ15M_0122_0223['Local time'] = pd.to_datetime(GJ15M_0122_0223['Local time'], dayfirst = True )
GJ15M_0122_0223 = GJ15M_0122_0223.sort_values(by = 'Local time')
GJ15M_0122_0223.reset_index(drop = True)

In [ ]:
GJ15M_0122_0223.tail(10)

###### 1st Half

In [ ]:
GJ15M_0122_0223_H1 = GJ15M_0122_0223.iloc[0:round(len(GJ15M_0122_0223)/2)].reset_index(drop = True)

In [ ]:
GJ15M_0122_0223_H1_test = TradingTest(GJ15M_0122_0223_H1,100)

In [ ]:
GJ15M_0122_0223_H1_test.gen_pivot(10,10)

In [ ]:
GJ15M_0122_0223_H1_test.gen_divSignal2(100,10)

In [ ]:
GJ15M_0122_0223_H1_test.test_falseBreakSig2MACD(ATR_surpass=0.3)

In [ ]:
GJ15M_0122_0223_H1_test.plot_PipReturn()

In [ ]:
GJ15M_0122_0223_H1_test.result_Sell

In [ ]:
a = 6752
GJ15M_0122_0223_H1_test.graph_Sample(a-100, a+25)

###### 2nd Half

In [ ]:
GJ15M_0122_0223_H2 = GJ15M_0122_0223.iloc[-round(len(GJ15M_0122_0223)/2):].reset_index(drop = True)

In [ ]:
GJ15M_0122_0223_H2_test = TradingTest(GJ15M_0122_0223_H2,100)

In [ ]:
GJ15M_0122_0223_H2_test.gen_pivot(10,10)

In [ ]:
GJ15M_0122_0223_H2_test.df[GJ15M_0122_0223_H2_test.df['local time'].dt.date == datetime.date(2022,8,22)]

In [ ]:
GJ15M_0122_0223_H2_test.df.loc[2031]

In [ ]:
a = 2070
GJ15M_0122_0223_H2_test.graph_Sample(a-200, a+20,CurrHigh=False, CurrLow= False)

In [ ]:
GJ15M_0122_0223_H2_test.gen_divSignal2(100,10)

In [ ]:
GJ15M_0122_0223_H2_test.test_falseBreakSig2MACD(ATR_surpass=0.3)

In [ ]:
GJ15M_0122_0223_H2_test.result_Sell

In [ ]:
GJ15M_0122_0223_H2_test.graph_Sample(3455 -100, 3455+20)

#### 2020

In [ ]:
GJ15M_2020_test = TradingTest(GJ15M_2020.reset_index().drop(columns = ['index']), pipRetMulti = 100)

In [ ]:
GJ15M_2020_test.gen_pivot(5,5)
GJ15M_2020_test.gen_divSignal2(30)
GJ15M_2020_test.df.divSignal2MACD.value_counts()

In [ ]:
GJ15M_2020_resultS, GJ15M_2020_statS = falsebreak_wDiv_test(GJ15M_2020_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)
GJ15M_2020_resultB, GJ15M_2020_statB = falsebreak_wDiv_test(GJ15M_2020_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)
GJ15M_2020_resultAll = GetResultAll(GJ15M_2020_resultS,GJ15M_2020_resultB)
GJ15M_2020_statAll = Get_statAll(GJ15M_2020_resultAll)

In [ ]:
Plot_csPipReturn(GJ15M_2020_resultAll,side = 'All',spread  = 2, return_bar= 'All')

In [ ]:
Plot_csPipReturn(GJ15M_2020_resultS, side = 'sell', spread = 2, return_bar = 'All')
Plot_csPipReturn(GJ15M_2020_resultB, side = 'buy', spread = 2, return_bar = 'All')

#### 2021

In [ ]:
GJ15M_2021_test = TradingTest(GJ15M_2021.reset_index().drop(columns = ['index']), pipRetMulti = 100)

In [ ]:
test = TradingTest(GJ15M_2021, pipRetMulti = 100)

In [ ]:
test.df

In [ ]:
test.df.columns

In [ ]:
GJ15M_2021_test.gen_pivot(5,5)
GJ15M_2021_test.gen_divSignal2(30)
GJ15M_2021_test.df.divSignal2MACD.value_counts()

In [ ]:
GJ15M_2021_resultS, GJ15M_2021_statS = falsebreak_wDiv_test(GJ15M_2021_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)
GJ15M_2021_resultB, GJ15M_2021_statB = falsebreak_wDiv_test(GJ15M_2021_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)
GJ15M_2021_resultAll = GetResultAll(GJ15M_2021_resultS,GJ15M_2021_resultB)
GJ15M_2021_statAll = Get_statAll(GJ15M_2021_resultAll)

In [ ]:
GJ15M_2021_statAll

In [ ]:
Plot_csPipReturn(GJ15M_2021_resultAll,side = 'All',spread  = 2, return_bar= 'All')

In [ ]:
dfpl = GJ15M_2021_resultAll
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl.index,y=dfpl['PipReturn_20bar_cumsum']), row = 1, col = 1)
# fig.add_scatter(x = GJ1H_18_23_resultAll.iloc[138:]['gmt time'], y= GJ1H_18_23_resultAll.iloc[138:]['PipReturn_20bar_cumsum'],mode="markers",
#                             marker=dict(size=4, color="MediumPurple"),name="pivot", row=2, col=1)
       
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

In [ ]:
GJ15M_2021_resultAll.iloc[313]

In [ ]:
GJ15M_2021[GJ15M_2021['Gmt time'] >= '2021-11-1']

In [ ]:
pd.set_option('display.max_rows', None)


In [ ]:
GJ15M_2021.loc[24170:]

In [ ]:
GJ15M_2021[(GJ15M_2021['Gmt time'] == '2021-11-18 23:30:00')]

In [ ]:
GJ15M_2021.loc[24260]

In [ ]:
GJ15M_2021_test.graph_init_Sample(20820,20920,RSIpp= False, RSI= False, height=2000)

In [ ]:
Plot_csPipReturn(GJ15M_2021_resultS, side = 'sell', spread = 2, return_bar = 'All')
Plot_csPipReturn(GJ15M_2021_resultB, side = 'buy', spread = 2, return_bar = 'All')

### 1 hour

In [ ]:
print(os.listdir("D:/Data/Forex/GBPJPY/"))

In [ ]:
GJ1H_15_23 = pd.read_csv("D:/Data/Forex/GBPJPY/DUKAS_GBPJPY_Candlestick_1_Hour_BID_01.01.2015-14.01.2023.csv")

In [ ]:
GJ1H_15_23['Gmt time'] = pd.to_datetime(GJ1H_15_23['Gmt time'])

In [ ]:
GJ1H_15_18 = GJ1H_15_23[GJ1H_15_23['Gmt time'] < '2019-1-1']
GJ1H_18_23 = GJ1H_15_23[GJ1H_15_23['Gmt time'] >= '2019-1-1']

#### OANDA_TDV 2021-2023

In [ ]:
GJ1H_21_23 = pd.read_csv("D:/Data/Forex/GBPJPY/OANDA_GBPJPY, 60, 0121_0223.csv")

In [ ]:
GJ1H_21_23_test = TradingTest(GJ1H_21_23.reset_index(drop = True), pipRetMulti = 100)

In [ ]:
GJ1H_21_23_test.gen_pivot(10,10)


In [ ]:
GJ1H_21_23_test.gen_divSignal2(100,10)


In [ ]:
GJ1H_21_23_test.df.divSignal2MACD.value_counts()

In [ ]:
GJ1H_21_23_test_resultS, GJ1H_21_23_test_statS = falsebreak_wDiv_test(GJ1H_21_23_test.df,side = 'sell', fwdCandle=10,
                                                             elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass= 0.3,
                                                             ATR_commpare_range = 0)
GJ1H_21_23_test_resultB, GJ1H_21_23_test_statB = falsebreak_wDiv_test(GJ1H_21_23_test.df,side = 'buy', fwdCandle=10,
                                                             elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass= 0.3,
                                                             ATR_commpare_range = 0)

In [ ]:
GJ1H_21_23_test_statS

In [ ]:
GJ1H_21_23_test_statB

In [ ]:
GJ1H_21_23_resultAll = GetResultAll(GJ1H_21_23_test_resultS,GJ1H_21_23_test_resultB)
GJ1H_21_23_statAll = GetStatAll(GJ1H_21_23_resultAll)

In [ ]:
GJ1H_21_23_statAll

In [ ]:
GJ1H_21_23_test_resultS

In [ ]:
GJ1H_21_23_test.graph_init_Sample(3662,3782, height = 1300)

#### 2015-2018

In [ ]:
GJ1H_15_18_test = TradingTest(GJ1H_15_18.reset_index().drop(columns = ['index']), pipRetMulti = 100)

In [ ]:
GJ1H_15_18_test.df

In [ ]:
GJ1H_15_18_test.gen_pivot(5,5)
GJ1H_15_18_test.gen_divSignal2(30)
GJ1H_15_18_test.df.divSignal2MACD.value_counts()

In [ ]:
GJ1H_15_18_resultS, GJ1H_15_18_statS = falsebreak_wDiv_test(GJ1H_15_18_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)
GJ1H_15_18_resultB, GJ1H_15_18_statB = falsebreak_wDiv_test(GJ1H_15_18_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)

In [ ]:
GJ1H_15_18_resultAll = GetResultAll(GJ1H_15_18_resultS,GJ1H_15_18_resultB)
GJ1H_15_18_statAll = Get_statAll(GJ1H_15_18_resultAll)

In [ ]:
Plot_csPipReturn(GJ1H_15_18_resultAll, side = 'All', spread = 2, return_bar = "All",figsize = (15,6))

In [ ]:
GJ1H_15_18_statAll

In [ ]:
GJ1H_15_18_resultAll.columns

In [ ]:
dfpl = GJ1H_15_18_resultAll.iloc[138:]
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl['gmt time'],y=dfpl['PipReturn_3bar_cumsum']), row = 1, col = 1)
# fig.add_scatter(x = GJ1H_15_18_resultAll.iloc[138:]['gmt time'], y= GJ1H_15_18_resultAll.iloc[138:]['PipReturn_20bar_cumsum'],mode="markers",
#                             marker=dict(size=4, color="MediumPurple"),name="pivot", row=2, col=1)
       
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

In [ ]:
datetime.date(2016,7,7) - datetime.date(2018,12,28)

In [ ]:
Plot_csPipReturn(GJ1H_15_18_resultS, side = 'sell', spread = 2, return_bar='All')
Plot_csPipReturn(GJ1H_15_18_resultB, side = 'buy', spread = 2, return_bar='All')


#### 2018-2023

In [ ]:
GJ1H_18_23_test = TradingTest(GJ1H_18_23.reset_index().drop(columns = ['index']), pipRetMulti = 100)

In [ ]:
GJ1H_18_23_test.df

In [ ]:
GJ1H_18_23_test.gen_pivot(5,5)
GJ1H_18_23_test.gen_divSignal2(30)
GJ1H_18_23_test.df.divSignal2MACD.value_counts()

In [ ]:
GJ1H_18_23_resultS, GJ1H_18_23_statS = falsebreak_wDiv_test(GJ1H_18_23_test.df,side = 'sell', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)
GJ1H_18_23_resultB, GJ1H_18_23_statB = falsebreak_wDiv_test(GJ1H_18_23_test.df,side = 'buy', 
                                                             elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass= 0.0)

In [ ]:
GJ1H_18_23_resultAll = GetResultAll(GJ1H_18_23_resultS,GJ1H_18_23_resultB)
GJ1H_18_23_statAll = Get_statAll(GJ1H_18_23_resultAll)

In [ ]:
Plot_csPipReturn(GJ1H_18_23_resultAll, side = 'All', spread = 2, return_bar = "All",figsize = (15,6))

In [ ]:
GJ1H_18_23_statAll

In [ ]:
dfpl = GJ1H_18_23_resultAll
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl['gmt time'],y=dfpl['PipReturn_20bar_cumsum']), row = 1, col = 1)
# fig.add_scatter(x = GJ1H_18_23_resultAll.iloc[138:]['gmt time'], y= GJ1H_18_23_resultAll.iloc[138:]['PipReturn_20bar_cumsum'],mode="markers",
#                             marker=dict(size=4, color="MediumPurple"),name="pivot", row=2, col=1)
       
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

In [ ]:
datetime.date(2016,7,7) - datetime.date(2018,12,28)

In [ ]:
Plot_csPipReturn(GJ1H_18_23_resultS, side = 'sell', spread = 2, return_bar='All')
Plot_csPipReturn(GJ1H_18_23_resultB, side = 'buy', spread = 2, return_bar='All')

#### --------------

In [ ]:
print(os.listdir("D:Data/Forex/GBPJPY/OANDA_GBPJPY, 15, 0922_0223.csv"))

In [ ]:
gbpjpy_df = pd.read_csv("D:Data/Forex/GBPJPY/OANDA_GBPJPY, 15, 0922_0223.csv")
gbpjpy_df.columns = ['Gmt time', 'open', 'high', 'low', 'close', 'Volume']
gbpjpy_test = TradingTest(gbpjpy_df)
gbpjpy_test.gen_pivot(5,5)

In [ ]:
gbpjpy_test.gen_divSignal2(30)

In [ ]:
gj_sell_result_dfpl, gj_sell_stat_df = falsebreak_wDiv_test(gbpjpy_test.df, side = 'sell', elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass=0.5)
gj_sell_stat_df

In [ ]:
gj_sell_result_dfpl['PipReturn_10bar_cumsum'] = (gj_sell_result_dfpl['PipReturn_10bar']+0.2).cumsum()
plt.plot(gj_sell_result_dfpl.index, -gj_sell_result_dfpl['PipReturn_10bar_cumsum'])

In [ ]:
gj_buy_result_dfpl, gj_buy_stat_df = falsebreak_wDiv_test(gbpjpy_test.df, side = 'buy', elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass=0)
gj_buy_stat_df

In [ ]:
gj_buy_result_dfpl['PipReturn_10bar_cumsum'] = (gj_buy_result_dfpl['PipReturn_10bar']-0.2).cumsum()
plt.plot(gj_buy_result_dfpl.index, gj_buy_result_dfpl['PipReturn_10bar_cumsum'])

## GBPUSD

In [ ]:
GU15M_0922_0223 = pd.read_csv("D:/Data/Forex/GBPUSD/OANDA_GBPUSD, 15, 0922_0223.csv")
GU15M_0922_0223_test = TradingTest(GU15M_0922_0223,10000)

In [ ]:
GU15M_0922_0223_test.gen_pivot(10,10)
GU15M_0922_0223_test.gen_divSignal2(100,10)

In [ ]:
GU15M_0922_0223_test.test_falseBreakSig2MACD(ATR_surpass=0.1)

In [ ]:
GU15M_0922_0223_test.result_Buy

In [ ]:
GU15M_0922_0223_test.graph_Sample(1776,1887)

## NZDJPY


### 15M

#### TradingView data

In [ ]:
NJ15M_0922_0223 = pd.read_csv("D:/Data/Forex/NZDJPY/OANDA_NZDJPY, 15, 0922_0223.csv")

In [ ]:
NJ15M_0922_0223_test = TradingTest(NJ15M_0922_0223,100)

In [ ]:
NJ15M_0922_0223_test.gen_pivot(10,10)

In [ ]:
NJ15M_0922_0223_test.gen_divSignal2(100,10)

In [ ]:
NJ15M_0922_0223_test.test_falseBreakSig2MACD(ATR_surpass = 0.2)

In [ ]:
NJ15M_0922_0223_test.result_Buy

In [ ]:
observed_candleID = 6376
NJ15M_0922_0223_test.graph_Sample(observed_candleID -100,observed_candleID+20)

## BTC-USD

### 15 Min

#### Trading View data

In [ ]:
BTCUSD15M_0822_0223 = pd.read_csv("D:/Data/Crypto/BTCUSD/COINBASE_BTCUSD, 15, 0822_0223.csv")
BTCUSD15M_0822_0223_test = TradingTest(BTCUSD15M_0822_0223,pipRetMulti=0.1)

In [ ]:
BTCUSD15M_0822_0223_test.gen_pivot(10,10)
BTCUSD15M_0822_0223_test.gen_divSignal2(100,10)

In [ ]:
BTCUSD15M_0822_0223_test.df.divSignal2MACD.value_counts()

In [ ]:
BTCUSD15M_0822_0223_test.test_falseBreakSig2MACD(ATR_surpass=0.3)

In [ ]:
BTCUSD15M_0822_0223_test.result_Buy

In [ ]:
a = 1957
BTCUSD15M_0822_0223_test.graph_Sample(a -80, a+ 20)

In [ ]:
BTCUSD15M_0822_0223_test.plot_PipReturn()

In [ ]:
BTCUSD15M_0822_0223_test.result_Sell

In [ ]:
compare_index = 9660
BTCUSD15M_0822_0223_test.graph_Sample(compare_index - 100, compare_index+25)

In [ ]:
BTCUSD15M_0822_0223_statS

In [ ]:
BTCUSD15M_0822_0223_statB

In [ ]:
BTCUSD15M_0822_0223_resultB

In [ ]:
dfpl = USOIL_1122_0123_resultAll
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl['time'],y=dfpl['PipReturn_3bar_cumsum']), row = 1, col = 1)    
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

## USOIL

### 15 Min

In [ ]:
print(os.listdir("D:/Data/Commodity/USOIL/FX_USOIL, 15, 010922_080323.csv))

In [ ]:
USOIL15M = pd.read_csv("D:/Data/Commodity/USOIL/FX_USOIL, 15, 010922_080323.csv")
USOIL15M_test = TradingTest(USOIL15M,pipRetMulti=100)

In [ ]:
compare_range = 10

In [ ]:
USOIL15M_test.gen_pivot(compare_range,compare_range)
USOIL15M_test.gen_divSignal2(compare_range*10,compare_range)

In [ ]:
USOIL15M_test.df.divSignal2MACD.value_counts()

In [ ]:
USOIL15M_010922_080323_TDview_saved = [USOIL15M_test]

In [ ]:
%store USOIL15M_010922_080323_TDview_saved
exit

In [ ]:
%store -d USOIL15M_010922_080323_saved                       

In [ ]:
USOIL15M_test.test_falseBreakSig2MACD(ATR_surpass= 0.3,div_signal = 'no',ATR_commpare_range = 1)

In [ ]:
USOIL15M_test.result_Sell

In [ ]:
a = 4728
USOIL15M_test.graph_Sample(a - 100, a+30)

In [ ]:
dfpl = USOIL15M_resultAll
fig = make_subplots()       
fig.append_trace(go.Scatter(x=dfpl['time'],y=dfpl['PipReturn_10bar_cumsum']), row = 1, col = 1)    
            
fig.update_layout(xaxis_rangeslider_visible= False, width= 1000 ,height= 600,margin=dict(l=50,r=50,b=100,t=100,pad=4))

## EURAUD

### 15 Min

#### Trading view data

In [ ]:
EURAUD_15M_df = pd.read_csv("D:Data/Forex/EURAUD/OANDA_EURAUD, 15 0822_0223.csv")
EURAUD_15M_test = TradingTest(EURAUD_15M_df, pipRetMulti= 10000)

In [ ]:
compare_range = 10

In [ ]:
EURAUD_15M_test.gen_pivot(compare_range,compare_range)

In [ ]:
EURAUD_15M_test.gen_divSignal2(compare_range*10,compare_range)

In [ ]:
EURAUD_15M_test.df.divSignal2MACD.value_counts()

In [ ]:
EURAUD_15M_0822_0223_TDVIEW = [EURAUD_15M_test]

In [ ]:
%store -d EURAUD_15M_0822_0223_TDVIEW                     

In [ ]:
%store EURAUD_15M_0822_0223_TDVIEW
exit

In [ ]:
a = EURAUD_15M_0822_0223_TDVIEW[0]

In [ ]:
a.test_falseBreakSig2MACD()

In [ ]:
EURAUD15M_0122_0622_saved_test = EURAUD15M_0122_0622_list[1]

In [ ]:
EURAUD15M_0122_0622_saved_test.test_falseBreakSig2MACD(ATR_surpass = 0)

In [ ]:
EURAUD_15M_test.result_Sell

In [ ]:
a = 6337

In [ ]:
EURAUD_15M_test.graph_Sample(a-100, a+20)

#### Dukas data

##### 2022

1H

In [ ]:
EURAUD15M_0122_0622 = pd.read_csv("D:Data/Forex/EURAUD/Dukas_EURAUD_Candlestick_15_M_BID_01.01.2022-30.06.2022.csv")
EURAUD15M_0122_0622['Local time'] = pd.to_datetime(EURAUD15M_0122_0622['Local time'],dayfirst=True)
EURAUD15M_0122_0622.sort_values(by = 'Local time',inplace=True)
EURAUD15M_0122_0622.reset_index(drop = True, inplace=True)

In [ ]:
EURAUD15M_0122_0622_test = TradingTest(EURAUD15M_0122_0622,10000)

In [ ]:
EURAUD15M_0122_0622_test.gen_pivot(10,10)

In [ ]:
EURAUD15M_0122_0622_test.gen_divSignal2(100)

In [ ]:
EURAUD15M_0122_0622_test.test_falseBreakSig2MACD(ATR_surpass=0)

In [ ]:
EURAUD15M_0122_0622_test.result_Buy

In [ ]:
a = 219
EURAUD15M_0122_0622_test.graph_Sample(a-100, a+20)

In [ ]:
EURAUD15M_0122_0622_best_param = {'HiLowRange':'yes', 'ATR_surpass':'0'}

In [ ]:
EURAUD15M_0122_0622_list = [EURAUD15M_0122_0622_best_param,EURAUD15M_0122_0622_test]

In [ ]:
%store EURAUD15M_0122_0622_list
exit

2H

In [ ]:
EURAUD15M_0722_1222 = pd.read_csv("D:Data/Forex/EURAUD/Dukas_EURAUD_Candlestick_15_M_BID_01.07.2022-30.12.2022.csv")
EURAUD15M_0722_1222['Local time'] = pd.to_datetime(EURAUD15M_0722_1222['Local time'],dayfirst=True)
EURAUD15M_0722_1222.sort_values(by = 'Local time',inplace=True)
EURAUD15M_0722_1222.reset_index(drop = True, inplace=True)

In [ ]:
EURAUD15M_0722_1222_test = TradingTest(EURAUD15M_0722_1222,10000)

In [ ]:
EURAUD15M_0722_1222_test.gen_pivot(10,10)

In [ ]:
EURAUD15M_0722_1222_test.gen_divSignal2(100)

In [ ]:
EURAUD15M_0722_1222_Dukas = [EURAUD15M_0722_1222_test]

In [ ]:
%store EURAUD15M_0722_1222_Dukas

In [ ]:
exit

In [ ]:
%store

In [ ]:
EURAUD15M_0722_1222_test.test_falseBreakSig2MACD(ATR_surpass=0)

In [ ]:
EURAUD15M_0722_1222_test.result_Sell

In [ ]:
EURAUD15M_0722_1222_test.df.iloc[10575:]

In [ ]:
a = 10576
EURAUD15M_0722_1222_test.graph_Sample(a-100,a+20)

## GBPAUD

In [ ]:
GBPAUD15M_0922_0223 = pd.read_csv("D:Data/Forex/GBPAUD/OANDA_GBPAUD, 15, 0922_0223.csv")
GBPAUD15M_0922_0223_test = TradingTest(GBPAUD15M_0922_0223,10000)

In [ ]:
GBPAUD15M_0922_0223_test.gen_pivot(10,10)
GBPAUD15M_0922_0223_test.gen_divSignal2(100,10)

In [ ]:
GBPAUD15M_0922_0223_test.test_falseBreakSig2MACD(ATR_surpass=0.1)

In [ ]:
GBPAUD15M_0922_0223_test.plot_PipReturn()

In [ ]:
GBPAUD15M_0922_0223_test.result_Sell

In [ ]:
a = 3107
GBPAUD15M_0922_0223_test.graph_Sample(a-100, a+30)

## EURCHF

In [ ]:
print(os.listdir("D:Data/Forex/EURCHF/OANDA_EURCHF, 15, 0822_0223.csv))

In [ ]:
EURCHF_15M_df = pd.read_csv("D:Data/Forex/EURCHF/OANDA_EURCHF, 15, 0822_0223.csv")
EURCHF_15M_test = TradingTest(EURCHF_15M_df, pipRetMulti= 10000)

In [ ]:
compare_range = 10

In [ ]:
EURCHF_15M_test.gen_pivot(compare_range,compare_range)

In [ ]:
EURCHF_15M_test.gen_divSignal2(compare_range*10,compare_range)

In [ ]:
EURCHF_15M_test.df.divSignal2MACD.value_counts()

In [ ]:
EURCHF_15M_resultS, EURCHF_15M_statS = falsebreak_wDiv_test(EURCHF_15M_test.df,
                                                                side = 'sell', elim_cons_trade = True,
                                                                falsebreak_bar= 0, ATR_surpass= 0,
                                                                ATR_commpare_range=0, fwdCandle=compare_range
                                                                )
EURCHF_15M_resultB, EURCHF_15M_statB = falsebreak_wDiv_test(EURCHF_15M_test.df,
                                                                side = 'buy', elim_cons_trade = True , 
                                                                falsebreak_bar= 0, ATR_surpass= 0,
                                                                ATR_commpare_range=0,fwdCandle=compare_range
                                                                )

In [ ]:
EURCHF_15M_resultS_copy = EURCHF_15M_resultS.copy()
EURCHF_15M_resultS_copy[pip_return_masking]  = EURCHF_15M_resultS_copy[pip_return_masking]*-1
EURCHF_15M_resultAll = pd.concat([EURCHF_15M_resultB, EURCHF_15M_resultS_copy])
EURCHF_15M_resultAll['time'] = pd.to_datetime(EURCHF_15M_resultAll['time'])
EURCHF_15M_resultAll = EURCHF_15M_resultAll.sort_values(by = "time")
EURCHF_15M_resultAll = EURCHF_15M_resultAll.reset_index().drop(columns = ['level_0'])
EURCHF_15M_StatAll = GetStatAll(EURCHF_15M_resultAll)

In [ ]:
EURCHF_15M_statS

In [ ]:
EURCHF_15M_statB

In [ ]:
EURCHF_15M_StatAll

In [ ]:
Plot_csPipReturn(EURCHF_15M_resultAll,side='All',spread=2, return_bar='All')

In [ ]:
EURCHF_15M_resultB

In [ ]:
EURCHF_15M_test.df.iloc[6071]

In [ ]:
EURCHF_15M_test.graph_init_Sample(9322,9452, height = 1500)

## GBPCAD

### 15 Min


In [ ]:
print(os.listdir("D:Data/Forex/GBPCAD/OANDA_GBPCAD, 15, 0822_0223.csv))

In [ ]:
GBPCAD_15M_df = pd.read_csv("D:Data/Forex/GBPCAD/OANDA_GBPCAD, 15, 0822_0223.csv")
GBPCAD_15M_test = TradingTest(GBPCAD_15M_df, pipRetMulti= 10000)

In [ ]:
compare_range = 10

In [ ]:
GBPCAD_15M_test.gen_pivot(compare_range,compare_range)

In [ ]:
GBPCAD_15M_test.gen_divSignal2(compare_range*10,compare_range)

In [ ]:
GBPCAD_15M_test.df.divSignal2MACD.value_counts()

In [ ]:
GBPCAD_15M_resultS, GBPCAD_15M_statS = falsebreak_wDiv_test(GBPCAD_15M_test.df,
                                                                side = 'sell', elim_cons_trade = True,
                                                                falsebreak_bar= 0, ATR_surpass= 0,
                                                                ATR_commpare_range=0, fwdCandle=compare_range
                                                                )
GBPCAD_15M_resultB, GBPCAD_15M_statB = falsebreak_wDiv_test(GBPCAD_15M_test.df,
                                                                side = 'buy', elim_cons_trade = True , 
                                                                falsebreak_bar= 0, ATR_surpass= 0,
                                                                ATR_commpare_range=0,fwdCandle=compare_range
                                                                )

In [ ]:
GBPCAD_15M_resultS_copy = GBPCAD_15M_resultS.copy()
GBPCAD_15M_resultS_copy[pip_return_masking]  = GBPCAD_15M_resultS_copy[pip_return_masking]*-1
GBPCAD_15M_resultAll = pd.concat([GBPCAD_15M_resultB, GBPCAD_15M_resultS_copy])
GBPCAD_15M_resultAll['time'] = pd.to_datetime(GBPCAD_15M_resultAll['time'])
GBPCAD_15M_resultAll = GBPCAD_15M_resultAll.sort_values(by = "time")
GBPCAD_15M_resultAll = GBPCAD_15M_resultAll.reset_index().drop(columns = ['level_0'])
GBPCAD_15M_StatAll = GetStatAll(GBPCAD_15M_resultAll)

In [ ]:
GBPCAD_15M_statS

In [ ]:
GBPCAD_15M_statB

In [ ]:
GBPCAD_15M_StatAll

In [ ]:
Plot_csPipReturn(XAU15M_121322_012723_resultAll,side='All',spread=2, return_bar='All')

In [ ]:
GBPCAD_15M_resultB

In [ ]:
GBPCAD_15M_test.df.iloc[6071]

In [ ]:
GBPCAD_15M_test.graph_init_Sample(9322,9452, height = 1500)

## EURUSD

#### EURUSD 5 min


In [ ]:
print(os.listdir("D:Data/Forex/EURUSD/FX_EURUSD, 15, 010922_080223.csv"))

In [ ]:
EU15M_0922_0223 = pd.read_csv("D:Data/Forex/EURUSD/FX_EURUSD, 15, 010922_080223.csv")
EU15M_0922_0223_test = TradingTest(EU15M_0922_0223,pipRetMulti=10000)

In [ ]:
compare_range = 10

In [ ]:
EU15M_0922_0223_test.gen_pivot(compare_range,compare_range)

In [ ]:
EU15M_0922_0223_test.gen_divSignal2(compare_range*10,compare_range)

In [ ]:
EU15M_0922_0223_test.df.divSignal2MACD.value_counts()

In [ ]:
EU15M_0922_0223_resultS, EU15M_0922_0223_statS = falsebreak_wDiv_test(EU15M_0922_0223_test.df,
                                                                side = 'sell', elim_cons_trade = True,
                                                                falsebreak_bar= 0, ATR_surpass= 0.1,
                                                                ATR_commpare_range=0, fwdCandle=compare_range
                                                                )
EU15M_0922_0223_resultB, EU15M_0922_0223_statB = falsebreak_wDiv_test(EU15M_0922_0223_test.df,
                                                                side = 'buy', elim_cons_trade = True , 
                                                                falsebreak_bar= 0, ATR_surpass= 0.1,
                                                                ATR_commpare_range=0,fwdCandle=compare_range
                                                                )

In [ ]:
EU15M_0922_0223_statS

In [ ]:
EU15M_0922_0223_statB

### VN30F1M

#### 5 min from 04/12/2022

In [ ]:
print(os.listdir("D:/Data/Stock/VN/Future/"))

In [ ]:
VN30F1M_5m_df = pd.read_csv("D:/Data/Stock/VN/Future/TDView_HNX_DLY_VN301!, 5.csv")

In [ ]:
VN30F1M_5m_df = VN30F1M_5m_df[['time', 'open', 'high', 'low', 'close','Volume']]

In [ ]:
VN30F1M_5m_tdtest = TradingTest(VN30F1M_5m_df, pipRetMulti= 1)

In [ ]:
VN30F1M_5m_tdtest.gen_pivot(5,5)
VN30F1M_5m_tdtest.gen_divSignal2(30)

##### Sell

In [ ]:
VN30_5m_sell_df, VN30_5m_sell_stats = falsebreak_wDiv_test(VN30F1M_5m_tdtest.df, side = 'sell', elim_cons_trade = True , falsebreak_bar = 1, ATR_surpass=0.1)
VN30_5m_sell_stats

In [ ]:
Plot_csPipReturn(VN30_5m_sell_df,'sell', 0.15, 3 )


##### Buy

In [ ]:
VN30_5m_buy_df, VN30_5m_buy_stats = falsebreak_wDiv_test(VN30F1M_5m_tdtest.df, side = 'buy', elim_cons_trade = True , falsebreak_bar= 1, ATR_surpass=0.1)
VN30_5m_buy_stats

In [ ]:
Plot_csPipReturn(VN30_5m_buy_df, 'buy', 0.15, 1)

In [ ]:
VN30_5m_buy_df['PipReturn_3bar_cumsum'] = (VN30_5m_buy_df['PipReturn_3bar'] - 0.15).cumsum()
plt.plot(VN30_5m_buy_df.index, VN30_5m_buy_df['PipReturn_3bar_cumsum'])

#### 15 min from 04/2020

In [ ]:
print(os.listdir("D:/Data/Stock/VN/Future/"))

In [ ]:
VN30F1M_15m_df = pd.read_csv("D:/Data/Stock/VN/Future/TDView_HNX_DLY_VN301!, 15.csv")

In [ ]:
VN30F1M_15m_test.df

In [ ]:
VN30F1M_15m_test = TradingTest(VN30F1M_15m_df, pipRetMulti=1)

In [ ]:
VN30F1M_15m_test.gen_pivot(5,5)
VN30F1M_15m_test.gen_divSignal2(30)

##### Sell

In [ ]:
VN30_15m_sell_df, VN30_15m_sell_stats = falsebreak_wDiv_test(VN30F1M_15m_test.df,
                                                             side = 'sell', elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass=0)
VN30_15m_sell_stats

In [ ]:
Plot_csPipReturn(VN30_15m_sell_df, side = 'sell', spread = 0.15, return_bar='All')

##### Buy

In [ ]:
VN30_15m_buy_df, VN30_15m_buy_stats = falsebreak_wDiv_test(VN30F1M_15m_test.df, side = 'buy', elim_cons_trade = True , falsebreak_bar= 0, ATR_surpass=0.)
VN30_15m_buy_stats

In [ ]:
VN30_15m_buy_df.tail(5)

In [ ]:
Plot_csPipReturn(VN30_15m_buy_df, side = 'buy', spread = 0.15, return_bar='All')

### AUDNZD

#### DUKAS_AUDNZD_Candlestick_5_M_BID_01.07.2022-14.01.2023

In [ ]:
print(os.listdir("D:/5.Data center/Forex/AUDNZD/"))

In [ ]:
AN_df = pd.read_csv("D:/5.Data center/Forex/AUDNZD/DUKAS_AUDNZD_Candlestick_5_M_BID_01.07.2022-14.01.2023.csv")
AN_df.columns = ['Gmt time', 'open', 'high', 'low', 'close', 'Volume']

In [ ]:
AN_tdtest = TradingTest(AN_df, pipRetMulti=10000)

In [ ]:
AN_tdtest.gen_pivot(5,5)
AN_tdtest.gen_divSignal2(30)

In [ ]:
AN_tdtest.df.divSignal2MACD.value_counts()

In [ ]:
AN_sell_df, AN_sell_stat = falsebreak_wDiv_test(AN_tdtest.df, side = 'sell', elim_cons_trade = True , falsebreak_bar=0, ATR_surpass=0)
AN_sell_stat

In [ ]:
AN_buy_df, AN_buy_stat = falsebreak_wDiv_test(AN_tdtest.df, side = 'buy', elim_cons_trade = True , falsebreak_bar=0, ATR_surpass=0)
AN_buy_stat

### EURGBP

#### DUKAS_EURGBP_Candlestick_5_M_BID_01.07.2022-14.01.2023

In [ ]:
EG_df = pd.read_csv("D:/5.Data center/Forex/EURGBP/DUKAS_EURGBP_Candlestick_5_M_BID_01.07.2022-14.01.2023.csv")

In [ ]:
EG_tdtest = TradingTest(EG_df, pipRetMulti=10000)

In [ ]:
EG_tdtest.gen_pivot(5,5)
EG_tdtest.gen_divSignal2(30)

In [ ]:
EG_tdtest.df.divSignal2MACD.value_counts()

In [ ]:
EG_sell_df,EG_sell_stat = falsebreak_wDiv_test(EG_tdtest.df, side = 'sell', elim_cons_trade = False , falsebreak_bar=1, ATR_surpass=0)
EG_sell_stat

In [ ]:
EG_buy_df,EG_buy_stat = falsebreak_wDiv_test(EG_tdtest.df, side = 'buy', elim_cons_trade = True , falsebreak_bar=1, ATR_surpass=0)
EG_buy_stat

In [ ]:
np.polyfit??

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1, row_heights=[0.7, 0.2,0.1])

fig.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
              row=1, col=1)

fig.add_trace(go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
              row=2, col=1)
fig.add_trace(go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
              row=3, col=1)

fig.show()

## Combination

### Tradingview data 0822-0223 

In [ ]:
%store

In [ ]:
%store -r USOIL15M_010922_080323_TDview_saved  
USOIL15M_010922_080323_TDview_test = USOIL15M_010922_080323_TDview_saved[0]
USOIL15M_010922_080323_TDview_test

In [ ]:
%store -r EURAUD_15M_0822_0223_TDVIEW                          
EURAUD_15M_0822_0223_TDVIEW_test = EURAUD_15M_0822_0223_TDVIEW[0]
EURAUD_15M_0822_0223_TDVIEW_test

In [ ]:
%store -r GJ15M_0822_0223_test_tdview_saved                    
GJ15M_0822_0223_TDview_test = GJ15M_0822_0223_test_tdview_saved[0]
GJ15M_0822_0223_TDview_test

In [ ]:
%store -r XAU15M_0822_0223_TDVIEW_test_saved                                  
XAU15M_0822_0223_TDVIEW_test = XAU15M_0822_0223_TDVIEW_test_saved[0]
XAU15M_0822_0223_TDVIEW_test


### With divergence

In [ ]:
ATR_surpass = 0.3
USOIL15M_010922_080323_TDview_test.test_falseBreakSig2MACD(ATR_surpass=ATR_surpass)
EURAUD_15M_0822_0223_TDVIEW_test.test_falseBreakSig2MACD(ATR_surpass=ATR_surpass)
GJ15M_0822_0223_TDview_test.test_falseBreakSig2MACD(ATR_surpass= ATR_surpass)
XAU15M_0822_0223_TDVIEW_test.test_falseBreakSig2MACD(ATR_surpass= ATR_surpass)

In [ ]:
# Eliminate BTCUSD due to high volatility
XAU_GJ_OIL_EA_15M_0822_0223 = pd.concat([USOIL15M_010922_080323_TDview_test.result_All, 
                                              EURAUD_15M_0822_0223_TDVIEW_test.result_All,
                                              GJ15M_0822_0223_TDview_test.result_All,
                                              XAU15M_0822_0223_TDVIEW_test.result_All])
XAU_GJ_OIL_EA_15M_0822_0223 = XAU_GJ_OIL_EA_15M_0822_0223.sort_values(by = 'time').reset_index(drop = True)
display(GetStatAll(XAU_GJ_OIL_EA_15M_0822_0223))
Plot_csPipReturn(XAU_GJ_OIL_EA_15M_0822_0223,side = 'All', spread = 2, return_bar = 'All')

### Without divergence

In [ ]:
ATR_surpass = 0.3
div_signal = 'no'
ATR_commpare_range = 1
USOIL15M_010922_080323_TDview_test.test_falseBreakSig2MACD(ATR_surpass=ATR_surpass,div_signal=div_signal,ATR_commpare_range = ATR_commpare_range)
EURAUD_15M_0822_0223_TDVIEW_test.test_falseBreakSig2MACD(ATR_surpass=ATR_surpass,div_signal=div_signal,ATR_commpare_range = ATR_commpare_range)
GJ15M_0822_0223_TDview_test.test_falseBreakSig2MACD(ATR_surpass=ATR_surpass,div_signal=div_signal,ATR_commpare_range = ATR_commpare_range)
XAU15M_0822_0223_TDVIEW_test.test_falseBreakSig2MACD(ATR_surpass=ATR_surpass,div_signal=div_signal,ATR_commpare_range = ATR_commpare_range)

In [ ]:
# Eliminate BTCUSD due to high volatility
XAU_GJ_OIL_EA_15M_0822_0223 = pd.concat([
                                              USOIL15M_010922_080323_TDview_test.result_All, 
                                              EURAUD_15M_0822_0223_TDVIEW_test.result_All,
                                              GJ15M_0822_0223_TDview_test.result_All,
                                              XAU15M_0822_0223_TDVIEW_test.result_All
                                        ])
XAU_GJ_OIL_EA_15M_0822_0223 = XAU_GJ_OIL_EA_15M_0822_0223.sort_values(by = 'time').reset_index(drop = True)
display(GetStatAll(XAU_GJ_OIL_EA_15M_0822_0223))
Plot_csPipReturn(XAU_GJ_OIL_EA_15M_0822_0223,side = 'All', spread = 2, return_bar = 'All')